<div style="
    background: rgba(25, 25, 25, 0.55);
    backdrop-filter: blur(16px) saturate(150%);
    -webkit-backdrop-filter: blur(16px) saturate(150%);
    border: 1px solid rgba(255, 255, 255, 0.12);
    border-radius: 18px;
    padding: 45px 30px;
    text-align: center;
    font-family: 'Inter', 'Segoe UI', 'Helvetica Neue', Arial, sans-serif;
    color: #e0e0e0;
    box-shadow: 0 0 30px rgba(0, 0, 0, 0.35);
    margin: 40px auto;
    max-width: 800px;
">

  <h1 style="
      font-size: 2.8em;
      font-weight: 700;
      margin: 0 0 8px 0;
      letter-spacing: -0.02em;
      background: linear-gradient(90deg, #00e0ff, #9c7eff);
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
  ">
      Machine Learning Project
  </h1>

  <h2 style="
      font-size: 1.6em;
      font-weight: 500;
      margin: 0 0 25px 0;
      color: #b0b0b0;
      letter-spacing: 0.5px;
  ">
      Cars 4 You - Predicting Car Prices
  </h2>

  <p style="
      font-size: 1.25em;
      font-weight: 500;
      color: #c0c0c0;
      margin-bottom: 6px;
  ">
      Group 5 - Lukas Belser, Samuel Braun, Elias Karle, Jan Thier
  </p>

  <p style="
      font-size: 1.05em;
      font-weight: 400;
      color: #8a8a8a;
      font-style: italic;
      letter-spacing: 0.5px;
  ">
      Machine Learning End Results · 22.12.2025
  </p>
</div>


**Table of Contents**

- [1. Import Packages and Data](#1-import-packages-and-data)  

  - [1.1 Import Required Packages](#11-import-required-packages)  

  - [1.2 Load Datasets](#12-load-datasets)  

  - [1.3 Kaggle Setup](#13-kaggle-setup)  

- [2. Data Preparation](#2-data-preparation)  

  - [2.1 Data Cleaning](#21-data-cleaning)  

  - [2.2 Data Imputation](#22-data-imputation)  

  - [2.3 Outlier Handling](#23-outlier-handling)  

  - [2.4 Missing Values Handling](#24-missing-values-handling)  

  - [2.5 Feature Engineering](#25-feature-engineering)  

  - [2.6 Encoding, Transforming and Scaling](#26-encoding-transforming-and-scaling)  

  - [2.7 Feature Selection](#27-feature-selection)  

  - [2.8 Create Final Preprocessing Pipeline](#28-create-final-preprocessing-pipeline)  

- [3. Model Assessment Strategy and Metrics](#3-model-assessment-strategy-and-metrics)  

- [4. Baseline Model Comparison](#4-baseline-model-comparison)  

  - [4.1 Setup Default Models (Original vs. Optimized Preprocessing)](#41-setup-default-models-original-vs-optimized-preprocessing)  

  - [4.2 Run the Models](#42-run-the-models)  

  - [4.3 Baseline Model Comparison Discussion of Results](#43-baseline-model-comparison-discussion-of-results)  

- [5. Hyperparameter Tuning of Top Candidates](#5-hyperparameter-tuning-of-top-candidates)  

  - [5.1 Tree-Based RandomForest](#51-tree-based-randomforest)  

  - [5.2 Tree-Based Extra Trees](#52-tree-based-extra-trees)  

- [6. Comparison of Tuned Models](#6-comparison-of-tuned-models)  

- [7. Deployment and Prediction on Test Set](#7-deployment-and-prediction-on-test-set)  

- [8. Visualizations and Insights of Best Model and the Data Preparation Pipeline](#8-visualizations-and-insights-of-best-model-and-the-data-preparation-pipeline)  

  - [8.1 Data Preparation Pipeline](#81-data-preparation-pipeline)  

    - [8.1.1 Start](#811-start)  

    - [8.1.2 Cleaning](#812-cleaning)  

    - [8.1.3 Unused Outlier Handling](#813-unused-outlier-handling)  

    - [8.1.4 Missing Values Handling](#814-missing-values-handling)  

    - [8.1.5 Feature Engineering](#815-feature-engineering)  

    - [8.1.6 Transformation, Scaling, Encoding](#816-transformation-scaling-encoding)  

    - [8.1.7 Feature Selection](#817-feature-selection)  

    - [8.1.8 Entire Pipeline with all outputs](#818-entire-pipeline-with-all-outputs)  

  - [8.2 Model](#82-model)  

- [9. Discussion and Outlook](#9-discussion-and-outlook)  

- [10. Open-Ended-Section](#10-open-ended-section)  

  - [10.1 SHAP Interpretability for Our Final Tree Model (Informative Only)](#101-shap-interpretability-for-our-final-tree-model-informative-only)  

  - [10.2 Global vs Brand- and Model-Specific Models](#102-global-vs-brand--and-model-specific-models)  

  - [10.3 Price Predictor Web Application](#103-price-predictor-web-application)  

  - [10.4 Deep Learning Experiment: Pre-Training vs Training from Scratch](#104-deep-learning-experiment-pre-training-vs-training-from-scratch)  

 

#### **Group Member Contribution**    
 
Jan (25%):
- EDA
- Pipeline structure (Data Split)
- Missing Values Handling
- Feature Engineering
- Encoding, Transforming and Scaling
- Feature Selection
- Model Assessment Strategy
- Baseline Model Comparison
- Hyperparameter Tuning of Top Candidates
- Comparison of Tuned Models
- Deployment and Prediction on Test Set
- Visualizations and Insights of the Data Preparation Pipeline
- Discussion and Outlook
 
Samu (25%):
- Pipeline chart and Introduction Markdowns
- Data Cleaning
- Outlier Handling
- Feature Engineering
- Encoding, Transforming and Scaling
- Model Assessment Strategy
- Baseline Model Comparison
- Deployment and Prediction on Test Set
- Open End: SHAP Feature Importance
- Open End: Brand-specific model comparison
 
Lukas (25%):
- EDA
- Missing Values Handling
- Feature Engineering
- Encoding, Transforming and Scaling
- Feature Selection
- Model Assessment Strategy
- Baseline Model Comparison
- Hyperparameter Tuning of Top Candidates
- Open End: Brand-specific model comparison
 
Elias (25%):
- Baseline Model Comparison
- Hyperparameter Tuning of Top Candidates
- Comparison of Tuned Models
- Deployment and Prediction on Test Set
- Visualizations and Insights of Best Model and the Data
- Open End: SHAP Feature Importance
- Open End: Analytical Interface
- Open End: Deep-Learning
 

#### Abstract

The Cars4You project aims to accelerate and standardize used-car price evaluations by replacing manual, subjective pricing with a production-ready machine learning pipeline. Our objective was to optimize predictive accuracy (MAE) on unseen cars while ensuring robustness to wrong inputs and a leakage-free evaluation.

Our EDA containing univariate, bivariate and multivariate analysis showed three dominant challenges: (1) inconsistent raw entries (typos, invalid ranges, sparse categories), (2) strong segmentation effects by brand/model, and (3) heavy-tailed numeric distributions (notably mileage). 

We addressed these with a custom engineered and reproducible sklearn pipeline. It follows the state of the art pipeline architecture and consists the following transformers: 

deterministic cleaning and category canonicalization with `CarDataCleaner` and hierarchical imputation with `IndividualHierarchyImputer`. We then added domain-informed feature engineering with `CarFeatureEngineer` to encode depreciation, usage intensity, efficiency/performance ratios, interaction effects, and relative positioning within brand/model segments.

Encoding and scaling were consolidated in a `ColumnTransformer` combining selective log transforms, `RobustScaler`, one-hot encoding, and median target encoding for high-signal categorical structure.

To reduce noise and improve generalization, we implemented automated feature selection as a dedicated pipeline stage: VarianceThreshold followed by majority voting across complementary selectors (Spearman relevance+redundancy, mutual information, and tree-based importance). SHAP was used strictly for interpretability and diagnostics in the end.

All model selection and tuning followed a consistent 5-fold cross-validation protocol. Primary evaluation metric MAE was set at the beginning of the project, we also evaluated RMSE and R2. After a first run of different models on original and engineered features, further hyperparameter tuning on the tree-based models HistGradientBoost and RandomForest was decided. The final tuned RF pipeline improved substantially over a naive median baseline (MAE ≈ 6.8k), achieving approximately **£1.2k MAE** in cross-validation.

For detailed methodological reasoning, trade-offs and further findings please refer to the respective sections.

Visual representation of the approach we followed in this notebook:
<img src="images/process_ML.png" alt="Drawing" style="width: 1000px;"/>

Visual representation of our technical pipeline:

<img src="images/pipeline-chart.png" alt="Drawing" style="width: 1000px;"/>

### 0. Import Packages and Data

#### 1.1 Import Required Packages

In [ ]:
!pip install kaggle
!pip install shap
!pip install -U scikit-learn
!pip install category_encoders
!pip install ydata-profiling

In [ ]:
import pandas as pd
import numpy as np
import os
 
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, StandardScaler, FunctionTransformer, RobustScaler
from category_encoders import QuantileEncoder
from category_encoders.wrapper import NestedCVWrapper
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
 
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt; plt.rcParams.update({"figure.max_open_warning": 0, "figure.dpi": 100})
import shap

import joblib
import pickle

from pipeline_functions import CarDataCleaner, IndividualHierarchyImputer, CarFeatureEngineer, DebugTransformer, MajorityVoteSelectorTransformer, MutualInfoThresholdSelector, SpearmanRelevancyRedundancySelector, create_model_pipe, get_cv_results, model_hyperparameter_tuning, SetOutputCompatibleWrapper
from visualization_functions import plot_selector_agreement, plot_train_val_comparison

#### 1.2 Load Datasets

In [ ]:
df_cars_train = pd.read_csv("train.csv").rename(columns={"Brand": "brand",
                                                        "paintQuality%": "paintQuality"})
df_cars_test = pd.read_csv("test.csv").rename(columns={"Brand": "brand",
                                                       "paintQuality%": "paintQuality"})

# Check for duplicates in carID column
print(f"Number of duplicate carIDs in training data: {df_cars_train['carID'].duplicated().sum()}")
print(f"Total rows in training data: {len(df_cars_train)}")

#### 1.3 Kaggle Setup

In [ ]:
# Folder containing kaggle.json (add kaggle.json api token)
os.environ['KAGGLE_CONFIG_DIR'] = "/Workspace/Users/X@novaims.unl.pt"

# Test
!echo $KAGGLE_CONFIG_DIR

### I. Identifying Business Needs

**Context**       
Cars 4 You is an online car-resale company that buys cars from many brands. Sellers submit car details online, and the company traditionally relies on a mechanic inspection before making a purchase offer. Company growth has increased waiting lists for inspections, which risks losing potential customers to competitors. The business need is therefore to **expedite pricing** by generating a reliable **pre-inspection price estimate** directly from user-provided inputs.

**Main goals**     
**Business goal:** provide a fast, consistent price estimate at intake to reduce inspection bottlenecks and improve conversion.

**ML goal:** train a supervised **regression** model to predict `price` (GBP) from features available at form submission time. Because `paintQuality%` is filled by a mechanic during evaluation, it is treated as **non-production** input and is excluded from the deployed prediction path.

**Overall process (end-to-end)**      
1. **Data intake:** use the 2020 training dataset (features + target `price`) to develop and validate models; use the provided test dataset (features only) for final predictions and Kaggle submission.
2. **EDA → preprocessing decisions:** identify data inconsistencies, missingness patterns, segmentation by brand/model, and heavy-tailed variables; translate insights into pipeline steps.
3. **Leakage-safe pipeline:** implement cleaning, outlier handling, hierarchical imputation, feature engineering, encoding/scaling, and feature selection as sklearn transformers inside a single `Pipeline` so that every step is fitted only on training folds.
4. **Model benchmarking & optimization:** compare candidate regressors on a consistent protocol; tune the most promising models; select the most generalizable pipeline based on validation performance.
5. **Deployment output:** fit the selected pipeline on the full training set and generate predictions for `test.csv` to produce the final submission `.csv`.

**Model assessment strategy:**     
We adopt **5-fold cross-validation (CV)** on the training set as the single assessment strategy used throughout benchmarking and tuning. The primary metric is **MAE** (business-interpretable error in GBP), with RMSE and R² reported as complementary diagnostics. The Kaggle test set functions as an **external holdout** for the final chosen pipeline (labels hidden; performance observed via leaderboard score).

For more details on the respective steps in the pipeline, refer to the specific part in the notebook below.

### II. Data Exploration and Preprocessing

**Data Exploration:** For the analysis of the original features including the consequences for preprocessing, refer to notebook `group05_exploratory_data_analysis.ipynb`. These insights are then used to clean and prepare the data. In addition we redid the EDA after each preprocessing step to visualize the impact and debug the pipeline. This process can be redone in Section 3.5.     

**Top 3 EDA Key Insights:**
- Multiple **logical inconsistencies** that will be handled in Data Cleaning (e.g. mileage < 0)
- Some features, especially mileage and the target price, are **right-skewed** -> log-transform
- The target has high **correlations** with year, mileage and engineSize while other features (previousOwners) have no correlation. There is also a high correlation between features (e.g. spearman of roughly -0.8 for mileage and year)

**Preprocessing:** The steps taken to clean and prepare the data based on exploration are described and justified in the following respective Subsections on Data Cleaning. The inner-workings of these steps are later visualized in Section 3.5

#### 2.1 Data Split

**Train and Val**: We use `Cross-Validation` in the `sklearn pipeline` on the available training data to make use of all data while validating different approaches.    
-> We fix the random states everywhere to ensure that all models use the same split to ensure a fair model comparison

**Test** Use external hold-out set from kaggle as final test set (remains completely unseen to avoid leakage)
-> An additional val set is therefore not necessary and would waste training data

Final setup:
1. **Training Set (n-1 folds from CV)**: Used to fit models.
2. **Validation Set (1 fold from CV)**: Used to evaluate performance of models and tune hyperparameters, detect overfitting. 
3. **Test Set (Kaggle)**: Used only once at the end of the entire process to evaluate final model performance. Not considered before to prevent leakage.



<u>Place in the pipe:</u> The split is decided here because the data has to be split before the preprocessing steps to avoid data leakage. All of the following steps are part of the sklearn pipeline while the CV is not an explicit part of the pipeline but rather the technique that calls the pipeline with its separate folds.

In [ ]:
# Create CV (shuffle to ensure randomness in splits, random_state to make it reproducible and comparable across models)
rs = 5
cv = KFold(n_splits=5, shuffle=True, random_state=rs)
# => This cv will be passed for hyperparameter tuning later when training the models

# Split features and target
X_train = df_cars_train.drop(columns='price')
y_train = df_cars_train['price']

**Our findings:**
- CV achieves better results than using a hold-out set

**Consequences/Interpretation:**
- Usage of all available data is better for the model than 'wasting' training data for a hold-out set

#### 2.2 Data Cleaning

**Our approach:**
- We `clean data inconsistencies` and data entry errors that we found in the EDA
- These columns will be `set to NaN` for that specific entry to not lose rows in the data due to removing
- Afterwards, these values will be imputed (see Section 2.3)

____

**Numerical Features**

| **Feature** | **Allowed thresholds** | **Reasoning** | **# filtered below threshold** | **# filtered above threshold** |
| :--- | :--- | :--- | :---: | :---: |
| `year` | 1886 to 2020 | The first automobile is dated to 1886 (Benz Patent Motor Car), so earlier values are implausible; the dataset is from 2020, so newer model years are logically impossible. [1] | 0 | 358 |
| `mileage` | ≥ 0 | Negative mileage is not possible. | 369 | - |
| `tax` | ≥ 0 | Negative tax is not possible. | 378 | - |
| `mpg` | 5 to 150 | Lower bound 5 mpg is a conservative “sanity floor” below the least-efficient passenger car on FuelEconomy.gov’s list (Bugatti Mistral at 9 mpg combined), so we avoid removing valid low-efficiency cars while filtering implausible entries. [2] Upper bound 150 is a pragmatic cap to reduce leverage from extreme values and potential metric-mixing (e.g., MPGe-style values are defined as an energy-equivalent MPG for plug-in vehicles). [3] Reference point for high-efficiency non-EVs: Prius variants are ~50–56 mpg combined on FuelEconomy.gov. [4] | 49 | 221 |
| `engineSize` | 0.1 to 12.7 | Practical bounds: kei-class cars are capped at 660cc (0.66L), giving a grounded “small production car” reference point. [4] Very large historical production engines reach ~12.763L (Bugatti Type 41 / Royale). [5] Lower bound reduced to 0.1L as a conservative data-sanity floor (primarily to remove obvious errors) while avoiding unnecessary loss of potentially valid small-displacement entries. | 264 | 0 |
| `paintQuality` | 0 to 100 | Percentage values must be between 0 and 100. | 0 | 367 |
| `previousOwners` | ≥ 0 | Negative owner counts are not possible. | 371 | - |
| `hasDamage` | - | Only 0 and NaN values in the data -> no thresholding | . | . |


[[1]: https://group.mercedes-benz.com/company/tradition/company-history/1885-1886.html [2]: https://www.fueleconomy.gov/feg/best-worst.shtml https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&baseModel=Prius&make=Toyota&path=1&srchtyp=ymm&year1=2020&year2=2020 [3]: https://www.epa.gov/greenvehicles/fuel-economy-and-ev-range-testing [4]: https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&baseModel=Prius&make=Toyota&path=1&srchtyp=ymm&year1=2020&year2=2020 [5]: https://www.motortrend.com/features/what-is-a-kei-car [6]: https://www.bugatti-trust.co.uk/bugatti-type-41/]

----

**Categorial Features**

We fix data entry noise (typos, truncations, inconsistent casing) in two stages:

**1) Deterministic canonicalization:**

We first apply **static mapping tables** (built from our EDA findings on the training data) that collapse variants into one canonical label

- normalize formatting: lowercase + trim whitespace  
- map known variants/typos → one canonical category  
  - example: `"AUDI"`, `"udi"`, `"Aud"` → `Audi`  
  - example: `"semi-aut"`, `"emi-auto"` → `Semi-Auto`

This step is **fully deterministic**.

**2) Strict validity check:**

After canonicalization, we apply a **valid-category filter**:

- for high-cardinality `model`, we accept only **known valid model names** (our static canonical model vocabulary)
- any model token not in that vocabulary is considered unreliable and is set to **NaN**
- this prevents “garbage categories” (rare or malformed strings) from becoming real categories and harming generalization

Examples of what gets rejected (→ NaN):
- truncations like `"scirocc"` (intended: `scirocco`)
- incomplete single-letter tokens like `"a"`, `"q"`, `"x"` without enough context

**3) Special-rule buckets for ambiguous 1-letter tokens:**

Some one-letter model tokens are ambiguous and must not be guessed:

- if `brand == Audi` and `model == "a"` → set to `a_unknown`
- if `brand == Audi` and `model == "q"` → set to `q_unknown`
- if `brand == BMW` and `model == "x"` → set to `x_unknown`

If the brand is missing and the token is one of `{a, q, x}`, we do **not** guess and force it to **NaN**.

**4) Conservative fuzzy rescue (only for values that are still missing):**

Only after steps (1)–(3), we run a very strict “rescue step” to recover obvious typos:

- applied **only where the value is still NaN**
- candidate choices are restricted to **valid canonical categories** (never invent new labels)
- acceptance requires very high similarity (strict cutoff) and/or a unique prefix match  
  - example: `"pum"` → `puma` (unique prefix, safe)
  - example: `"sl clas"` → `sl class` (high similarity, safe)
- if no safe match exists, the value remains **NaN**

**5) Final handling of remaining missing categories:**

Any categorical value that is still missing after all steps remains **NaN** and is filled later by our **Imputer** (Section 2.3).

_____

**Leakage safety:**
All “data-driven vocabularies” used in the cleaner are learned inside `fit()` on the training fold only, so the pipeline remains leakage-safe under CV:
- the validation fold is never used to decide which categories are “valid”
- the same cleaning logic is applied consistently in train/validation/test
- the static mapping includes misspellings independent of the training fold and is included for every fold to prevent unnecessary pitfalls in model learning

In [ ]:
# Visualize the data cleaning by running it on raw df and inspect uniques
cleaner = CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)

df_cars_train_clean = cleaner.fit_transform(df_cars_train)
df_cars_test_clean  = cleaner.transform(df_cars_test)

# Print unique values for visualization after cleaning (more detailed inspection before and after Cleaning can be found in the EDA)
print("CLEANED TRAIN uniques")
for col in df_cars_train_clean.columns:
    print(col, df_cars_train_clean[col].unique())

**Our findings:**     
The findings are already included in the table above for easier overview and direct comparison
    - 'Number of filtered values below threshold'
    - 'Number of filtered values above threshold'

==> In total, [TODO] values are identified as data errors in the available training data and are set to NaN

**Consequences/Interpretation:**     
Handling data erros is crucial for effective model training. To identify these data erros, an extensive EDA is inevitable.

#### 2.3 Outlier Handling

**Practical considerations:**

We tried different outlier handling techniques (see `unused_experiments.ipynb`) but decided to not use an explicit technique because they hurt our performance.
Beside this explicit outlier handling between Cleaning and Imputation, we use the [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html) for scaling the features which handles outliers by centering and scaling the data based on the median and IQR (refer to Section 2.6).

<u>Place in the pipe</u>: 
- Before imputation to use original distribution for identifying the outliers (otherwise we would inflate the distributions with the imputed values)
- Then in imputation, fill the original gaps based on a distribution that does not includes the massive outliers (skewing the mean/median)    
  -> kill the outliers first (set to NaN) so the imputation for everyone becomes cleaner

**Findings**:      

As mentioned, using outlier handling significantly **hurt our best MAE**. The tried techniques are descripted in detail in the `unused_experiments.ipynb` file.

**Consequences/Interpretation:**   

Due to performance reasons we decided to use **no explicit outlier handling**.      
The worse performance due to outlier handling can be explained by the **importance of extreme cars for the algorithm**. When winsorizing or imputing these extreme values, we remove valuable signals. The specific decision to not use outlier handling is based on our best performing tree-based models which are not as sensitive to outliers like other models. When using other approaches that are more sensitive to extreme values, outlier handling can indeed be a valuable technique to improve performance.     

#### 2.4 Missing Values Handling

**Inidividual Hierarchy per Feature**

To impute the NaNs in each cell, an **individual hierarchical approach** is applied for each feature. For this, we compute group statistics (median and mode) of features that are highly correlated with the feature in the missing cell. If one of the required features is not available, we fallback to the next level until finally the global dataset statistic (median or mode) is used as a last fallback.    

Examples (for an extensive explanation refer to the IndividualHierarchyImputer class in pipeline_functions.py):
- `brand`: impute the brand using the model and if the model is NaN too, we impute using the grouped mode of (fuelType, transmission) because we identfied interaction between these two features and brand in the EDA.   
- `mileage`: we impute mileage with the median of the year because of high spearman corr identified in the EDA (~0.8).

The group statistics are only computed on the respective train folds and transformed on the val set to prevent leakage.   
-> When refitting the entire model, the entire train set is used to fit and the kaggle test set is transformed using the fitted values

<u>Place in the pipe:</u> The Imputation is decided here because the data has to be imputed on original values before engineering new features

**Our findings:**       
The IndividualHierarchyImputer improves performance over the best of our other tried imputers SimpleImputer (mode and median), KNN and the custom GroupImputer by ~30 MAE on the best baseline models.

**Consequences/Interpretation:**       
The improved performance over SimpleImputer and the custom GroupImputer show that each feature is **best approximated by a different feature (combination)**. There is **no golden rule** of imputing everything by the median or mode of the model. Instead, the **interaction** (e.g. spearman corr) of the missing feature and the other features should be regarded (e.g. impute mileage with median of year because of high spearman corr).

#### 2.5 Feature Engineering

**Our approach:**
- We implement feature engineering as an sklearn transformer (`CarFeatureEngineer`) **inside the pipeline**.
  - This makes the process **CV-safe / leakage-free**: all fold-specific statistics (e.g., model frequency, mean ages) are learned only on the training fold in `fit()` and applied to the validation fold in `transform()`.
- Important design notes: Interaction features use `(age + 1)` to avoid division by zero for cars in the reference year.
- We engineer features with two goals:
  1. **Inject domain structure** (age, usage intensity, efficiency, “big engine + old car” effects).
  2. **Create stronger signals for models** by expressing ratios and interactions that are difficult to learn reliably from raw variables.

**Input columns** (after cleaning + imputation):
- Numeric: `year`, `mileage`, `tax`, `mpg`, `engineSize`, `previousOwners`
- Categorical: `brand`, `model`, `transmission`, `fuelType`
- Boolean: `hasDamage`

---

| **New Feature** | **Calculation** | **Nature** | **Reasoning** |
| :--- | :--- | :--- | :--- |
| `age` | `ref_year - year` | Base | Captures depreciation; turns a calendar value into a meaningful pricing variable. |
| `mpg_x_engine` | `mpg * engineSize` | Interaction (product) | Joint signal for “performance vs efficiency” patterns (high engine + low mpg vs small engine + high mpg). |
| `engine_x_age` | `engineSize * (age + 1)` | Interaction (product) | Differentiates large engines in older cars vs newer cars; helps model capture age-dependent valuation of engine size. |
| `mpg_x_age` | `mpg * (age + 1)` | Interaction (product) | Captures age-dependent fuel-efficiency patterns (e.g., older fleets / technology differences) that can correlate with price. |
| `tax_x_age` | `tax * (age + 1)` | Interaction (product) | Models that tax effects can differ by car age (policy/regime + car segment composition). |
| `miles_per_year` | `mileage / (age + 1)` | Interaction (ratio) | Normalizes mileage by lifetime: 60k miles on a 3-year car is very different from 60k on a 10-year car; reduces collinearity between `mileage` and `age`. |
| `tax_per_mpg` | `tax / mpg` | Interaction (ratio) | “Cost pressure” proxy: high tax relative to efficiency can reflect segment / running cost patterns. |
| `engine_per_mpg` | `engineSize / mpg` | Interaction (ratio) | Performance-style signal: high engine with low mpg tends to indicate sporty/luxury configurations. |
| `brand_fuel` | `brand + "_" + fuelType` | Interaction (categorical) | Creates configuration groups for target encoding (e.g., Diesel BMW differs from Petrol BMW). |
| `brand_trans` | `brand + "_" + transmission` | Interaction (categorical) | Creates configuration groups for target encoding (e.g., Automatic Mercedes vs Manual Mercedes). |
| `model_freq` | `P(model)` from training fold | Popularity | Approximates market supply/demand stability: common models have more stable pricing; learned CV-safe in `fit()`. |
| `age_rel_brand` | `age - mean_age(brand)` | Relative / group-stat | Measures whether a car is newer/older than typical within its brand (brand-relative positioning). |
| `age_rel_model` | `age - mean_age(model)` | Relative / group-stat | Measures whether a car is newer/older than typical within its model (model-relative positioning). |
| `engine_rel_model` | `engineSize / mean_engineSize(model)` | Relative / group-stat | Captures whether a car is under-/over-engined relative to its model’s typical configuration. |

---

Legend (feature “Nature”)

- **Base Features**: derived from a single original variable (e.g. `age` from `year`)
- **Interaction Features**: combine multiple variables to capture non-additive effects
  - products (“amplifiers”) and ratios (“normalizers”)
- **Popularity Features**: learned from the training fold distribution (e.g. model frequency)
- **Relative / Group-stat Features**: compare a car to typical peers within `brand` or `model`
  - learned in `fit()` and applied in `transform()` to avoid leakage

---

Relation to encoding (Target Encoded Features)

We also create categorical “group keys” (`brand_fuel`, `brand_trans`) specifically so that our later target-encoding step inside the preprocessing pipeline can learn stable, configuration-specific signals.  
This encoding is handled **after** feature engineering and is **CV-safe** because it is part of the pipeline.

---


**Our findings:**
The engineered features are main drivers for performance improvement. In addition, to the improved MAE, the following findings support this insight:
- The correlation of the engineered features with the target show that we engineered meaningful features (e.g. miles_per_year: spearman corr of TODO, while miles and age only have Y and Z). This is visible through using y-data-profiling in the debugging pipeline.
- The feature importance of the engineered features in the final model (e.g. mpg_x_age: TODO)
- All Feature Importances and therefore also the impact of the engineered features in analyzed in detail in the Open End Section

| Feature | Impact |
| :--- | :--- |
| age | ... |


**Consequences/Interpretation:**

Some relationships are better captured through engineered features than raw original features. For example miles_per_year captures the actual usage of the car.


#### 2.6 Encoding, Transforming and Scaling

**Our approach:**
- We combine the different treatments for different natures of features in the ColumnTransformer
    - Numerics vs. Booleans vs. Categoricals
- We have one `baseline pipe` and one `optimized pipe` to compare basic preprocessing to optimized preprocessing
    - The baseline pipe does the bare minimum for the algorithms to work cleanly
    - The optimized pipe was adjusted iteratively through multiple experiments and trials during the process to optimize model performance

**Optimized Pipe:**
- `Log-Transforming` using [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) for skewed Numerics identified in the EDA
- `RobustScaler` ([docu](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)) for Numerics because it performed better than [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)     
    -> Scaling only on training data to avoid data leakage and then scale val and later test set with the fitted scaler of the training set     
- `Encoding` for Categoricals:
    - Low cardinality: [OHE](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) because of optimized performance on best models. We don't use drop='first' let the FS handle it (better performance for our best tree-based models).
    - High Cardinality: [Median TE](https://contrib.scikit-learn.org/category_encoders/quantile.html) and [Mean TE](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html) for optimized performance on best models
 
| **Feature** | Nature | Transformation | Encoding | Scaling |
| :--- | :--- | :--- | :--- | :--- |
| age | Numerical | - | - | Robust |
| tax | Numerical | - | - | Robust |
| mpg | Numerical | - | - | Robust |
| engineSize | Numerical | - | - | Robust |
| previousOwners | Numerical | - | - | Robust |
| mpg_x_engine | Numerical | - | - | Robust |
| engine_x_age | Numerical | - | - | Robust |
| mpg_x_age | Numerical | - | - | Robust |
| tax_x_age | Numerical | - | - | Robust |
| engine_per_mpg | Numerical | - | - | Robust |
| tax_per_mpg | Numerical | - | - | Robust |
| model_freq | Numerical | - | - | Robust |
| age_rel_brand | Numerical | - | - | Robust |
| age_rel_model | Numerical | - | - | Robust |
| engine_rel_model | Numerical | - | - | Robust |
| mileage | Numerical | log1p | - | Robust |
| miles_per_year | Numerical | log1p | - | Robust |
| hasDamage | Boolean | - | - | - |
| transmission | Categorical | - | OHE | - |
| fuelType | Categorical | - | OHE | - |
| brand | Categorical | - | Mean TE **and** Median TE | Robust |
| model | Categorical | - | Mean TE **and** Median TE | Robust |
| brand_fuel | Categorical | - | Mean TE **and** Median TE | Robust |
| brand_trans | Categorical | - | Mean TE **and** Median TE | Robust |
 
 
 
All operations are combined in a [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) which applies the different steps to the columns of the data in one unified pipeline (reproducible and prevents leakage)    
  -> outputs a combined feature matrix
 

In [ ]:
# Original features
orig_numeric_features = ["year", "mileage", "tax", "mpg", "engineSize", "previousOwners"]
orig_boolean = ["hasDamage"]
orig_categorical_features = ["brand", "model", "transmission", "fuelType"]


# Original and Engineered Features
numeric_features = [
    "age", "tax", "mpg", "engineSize", "previousOwners",        
    "mpg_x_engine", "engine_x_age", "mpg_x_age", "tax_x_age",   
    "engine_per_mpg", "tax_per_mpg",                            
    "model_freq",
    "age_rel_brand", "age_rel_model", "engine_rel_model"
]
numeric_features_for_log = ["mileage", "miles_per_year"]
boolean_features = ["hasDamage"]
categorical_features_ohe = ["transmission", "fuelType"]
categorical_features_te_mean = ["brand", "model", "brand_fuel", "brand_trans"]
categorical_features_te_median = ["brand", "model", "brand_fuel", "brand_trans"]
unused_columns = ["year"] # replaced by age (see Section on Feature Selection for details)

all_feature_names_before_encoding = set(numeric_features + numeric_features_for_log + boolean_features + categorical_features_ohe + categorical_features_te_median)
print(len(all_feature_names_before_encoding))

In [ ]:
enc_transf_scale = ColumnTransformer([
    ("log", Pipeline([
        ("log", FunctionTransformer(np.log1p, validate=False, feature_names_out="one-to-one")),  # log1p handles zeros safely
        ("scaler", RobustScaler()),
    ]), numeric_features_for_log),

    ("num", RobustScaler(), numeric_features),

    ("boolean", "passthrough", boolean_features),

    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features_ohe), # Use sparse_output=False to get dense array back (e.g. necessary for hgb)

    ("mean_te", Pipeline([ 
        ("encoder", TargetEncoder(target_type='continuous', cv=5, smooth='auto', random_state=rs)),
        ("scaler", RobustScaler()),
    ]), categorical_features_te_mean),

    # Smoothing (m) mitigates but doesnt eliminate leakage, so we use nested cv to work similar to the sklearn TE
    ("median_te", Pipeline(steps=[
        ('median_encoder', SetOutputCompatibleWrapper(NestedCVWrapper(QuantileEncoder(quantile=0.5, m=10.0), cv=cv, random_state=rs))), # not specifying the cols means it encodes all columns
        ('scaler', RobustScaler()),
    ]), categorical_features_te_median)
])

#### 2.7 Create Preprocessing Pipeline

**Our approach:**
- The [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) combines all steps into the preprocessing pipe (see table below)
- Through calling the pipeline for data preparation, we ensure that the data is preprocessed independently for each training fold    
-> prevent leakage while filling missing values, scaling, encoding, etc.

##### Summary of the pipelines before modeling: Baseline vs Optimized (including outliers)

| | **Baseline** | **Optimized** |
| :--- | :--- | :--- |
| **Data cleaning** | - | `CarDataCleaner` (Section 2.2) |
| **Outlier handling** | - | - |
| **Imputation** | SimpleImputer median/mode <br>(simplicity; median more robust than mean)</br> | `IndividualHierarchyImputer` (Section 2.4) |
| **Feature engineering** | - | `CarFeatureEngineer` (Section 2.5) |
| **Transformation** | - | Log-Transform selected skewed numerics (Section 2.6) |
| **Scaling** | StandardScaler <br>(popularity)</br>| RobustScaler (Section 2.6) |
| **Encoding** | OneHotEncoder <br>(simplicity, most straight-forward)</br>| OneHotEncoder + Target Encoding (Section 2.6) |
| **Feature selection** | - | VT + `Majority voting` (added in Section 3.2) |

In [ ]:
preprocessor_orig = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), orig_numeric_features),
    ("bool", SimpleImputer(strategy="most_frequent"), orig_boolean),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) # dont use drop='first' for tree models, use sparse_output=False to get dense array back (e.g. necessary for hgb)
    ]), orig_categorical_features)
])

In [ ]:
### The entire data preparation pipeline is build incorporating the enc_transf_scale defined in Section 2.6
preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ("ct", enc_transf_scale),
    # [Feature Selection: see Section 3.2 for details]
    # [Modeling: see Section 3.3 for details]
])

### III. Regression Benchmarking

#### 3.1) Model assessment strategy and metrics

**Performance Metrics**    
Comparison on train and val set to identify potential overfitting:
- **MAE** (Mean Absolute Error):     
    Used as primary metric because it is the metric used for evaluating in the Kaggle competition. It is easy to interpret in pounds.

- **MAE Std** (Standard Deviation):    
    Used to asses the variance of MAE across folds. For the secondary metrics the std will not be regarded as one std is enough to get a first understanding.

- **RMSE** (Root Mean Squared Error):    
    Used to identify large prediction errors which helps us understand model weaknesses (sensitive to outliers)

- **R-squared**:     
    Used to compare how well the models explain the underlying data compared to the mean (R-squared=0). It serves as an additional information compared to the two "error-metrics" MAE and RMSE.

=> All comparisons are based on **identical cross-validation splits** (same seed/folds). Reported values are fold-averaged metrics on train and validation.

**Structured approach to identify production model**
1. Run Baseline Models with default parameters to get a first glance of their performance
2. Compare and discuss model performance
3. Select top candidates for further optimization
4. Run Hyperparameter tuning on top candidates (Section 5)
5. Select best model and train it with best hyperparameters on minimizing the absolut_error (Section 5)

#### 3.2) Feature Selection Strategy and Results

**Strategy:**     

We apply an automatic feature selection approach in addition to the previously removed features (data cleaning, feature engineering)
- year: dropped because replaced by derived feature 'age'
- paintQuality: dropped because filled by mechanic so not available for our predictions in production  as the car prediction skips the mechanic

The goal is to create a very robust feature selection approach that finds features that are most likely actually irrelevant/redundant and therefore generate noise in the model that might lead to overfitting.     
To achieve that goal, we apply **two steps** inside the feature selection:
1) `Variance Threshold` (Filter) to filter constant variables ([docu](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html))    
(If using a different value than threshold 0, VarianceThreshold has to be applied before scaling, because e.g. standard scaling leads to std=1)
2) Majority voter:
    - `Spearman` handles the clean, obvious trends and cleans up redundancy ([docu](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html))
    - `MI` catches more complex relations that Spearman misses ([docu](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html))
    - `RF feature importance` to account for importance of the features ([docu](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#selectfrommodel))


The different voters capture different aspects:
| **Voter** | **Nature** | **Role & Responsibility** |
| :--- | :--- | :--- |
| **Spearman Voter** <br> *(SpearmanRedundancySelector)* | Filter | **Linear/Monotonic**<br>Captures obvious, strong relationships (e.g., "Newer cars are expensive"). Also handles Redundancy by filtering out features that are exact duplicates of better ones (Maximum Relevance, Minimum Redundancy (mRMR)-style pruning).|
| **MI Voter** <br> *(MutualInfoThresholdSelector)* | Filter | **Non-Linear**<br>Captures complex "physics" and non-monotonic patterns that correlation misses. |
| **RF Voter** <br> *(SelectFromModel)* | Embedded | **Interactions**<br>Captures features that are only important in combination with others. |

==> The feature selection is performed inside the pipelines cross-validation and consistent across all models, ensuring no data leakage and consistent feature selection logic.

<u>Place in the pipe:</u> The Feature Selection is placed after the scaling to have the features on one scale (just like in the lab)

In [ ]:
# Build the Pipeline including the fs_pipe
fs_pipe = Pipeline([
        ("vt", VarianceThreshold(threshold=0.0)),
        ('selector', MajorityVoteSelectorTransformer(
            selectors=[
                SpearmanRelevancyRedundancySelector(relevance_threshold=0.05, redundancy_threshold=0.95), # If we set redundancy threshold to 1.01, this becomes similar to just relevance filtering,
                MutualInfoThresholdSelector(threshold=0.01, n_neighbors=10), # Increasing n_neighbors makes the estimation more stable but computationally slower
                SelectFromModel(RandomForestRegressor(n_estimators=100, max_depth=8, n_jobs=-1, random_state=rs), # max_depths not too low (miss interactions) and not too high (selecting noise -> overfitting)
                                threshold='0.001*mean'), # threshold relative because it sums to 1 and if we have many features, many features will have a low importance but are still important],
                # [Unused] RFE unused because of high computational cost
                # rfecv_rf = RFECV(estimator = RandomForestRegressor(n_jobs=-1, max_depth=50), step=1, random_state=rs, cv=cv, scoring='neg_mean_absolute_error', min_features_to_select=5)
            ],
            min_votes=2))
        ])


# The entire data preparation pipeline is build incorporating the enc_transf_scale and fs-pipe defined in Section 2.6 and 3.2 respectively
preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ("ct", enc_transf_scale),
    ("fs", fs_pipe),
])

# Save preprocessor for reuse in DL experiments
with open('preprocessor_pipe.pkl', 'wb') as f:
    pickle.dump(preprocessor_pipe, f)

The following plot shows the votes of each contributor, resulting in the final decision whether to keep the feature or not. Be aware that the constant feature `boolean__hasDamage` has already bin pre-filtered by the VT before the majority voter.

**Argumentation** why we keep the certain features:     

In [ ]:
# Train Debug pipeline to visualize feature selection agreement
enc_transf_scale.set_output(transform="pandas")
fs_pipe.set_output(transform="pandas")
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020, verbose=False)),
    ("ct", (enc_transf_scale)),
    ("fs", (fs_pipe)),
])

# We call fit_tranform here on the entire training data of the debug preprocesor to just visualize the result of a final model using this FS
# -> The insights from here are not used for model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

# Feed the feed names after VT because VT is applied before the majority voting to remove constant features
feature_names_after_vt = debug_preprocessor_pipe.named_steps['fs'].named_steps['vt'].get_feature_names_out()
plot_selector_agreement(
    majority_selector = debug_preprocessor_pipe.named_steps['fs'].named_steps['selector'], 
    feature_names = feature_names_after_vt
)

**Our findings:**
- While trees are comparatively robust to unnecessary features, applying the feature selection pipeline improves the performance
- Using feature selection compared to not using feature selection on the exact same pipeline improved the performance by roughly 2 MAE on the best baseline models (ET and RF) while the score of linear regression stays the same

**Consequences/Interpretation:**
- The similar performance with or without feature selection (small Ablation Study) is due to the few selected features to remove (see Section 8.1.7)
- It shows that most features are important for the models while the feature selector detects useless and noisy features reliably (e.g. hasDamage and the OHE features that were created without dropping the first dummy: cat__transmission_Other and cat__fuelType_Other)

#### 3.3) Baseline Model Comparison

`Log-transforming the target (price)` using [TransformedTargetRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html) because EDA showed that it is heavily right-skewed. The model predicts the log-price and automatically convert it back to pounds at the end.

##### 3.3.1) Setup Default Models (Original vs. Optimized Preprocessing)

**Fair comparison:**    
- We use **default parameters** to get a first result of models potential to decide on which ones to use for further optimizing (hyperparameter tuning). - Only use the same random_state for reproducibility and n_jobs to speed up computations.    
- Use the rule of thumb provided in the lab for the MLP ()
- Baseline: DummyRegressor using the median price as prediction

The **log-transform** of the target is performed here because it is the most straightforwared implementation using the TransformedTargetRegressor ([docu](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html)). It handles transformation and afterwards uses the inverse automatically.

In [ ]:
### Median ###
baseline_median_pipe_orig = create_model_pipe(preprocessor_orig, DummyRegressor(strategy="median")) # Preprocessing does not matter for median

### Linear Models ###
linear_reg_default = LinearRegression()
linear_reg_default = TransformedTargetRegressor(
    regressor=linear_reg_default,
    func=np.log1p,
    inverse_func=np.expm1
)
linear_reg_pipe_orig = create_model_pipe(preprocessor_orig, linear_reg_default)
linear_reg_pipe_adjusted = create_model_pipe(preprocessor_pipe, linear_reg_default)


### Instance-Based ###
knn_default = KNeighborsRegressor(n_jobs=-3)
knn_default = TransformedTargetRegressor(
    regressor=knn_default,
    func=np.log1p,
    inverse_func=np.expm1
)
knn_pipe_orig = create_model_pipe(preprocessor_orig, knn_default)
knn_pipe_adjusted = create_model_pipe(preprocessor_pipe, knn_default)
# Long Duration (~4min)
# => Better performance than linear models but still worse than tree-based models -> not further optimized


### Neural Networks ###
# The number of hidden neurons should be between the size of the input layer (~10 for orig, ~25 for optimized) and the size of the output layer (1)
mlp_default = MLPRegressor(hidden_layer_sizes=(9,) , random_state=rs)
mlp_default = TransformedTargetRegressor(
    regressor=mlp_default,
    func=np.log1p,
    inverse_func=np.expm1
)
mlp_pipe_orig = create_model_pipe(preprocessor_orig, mlp_default)
mlp_pipe_adjusted = create_model_pipe(preprocessor_pipe, mlp_default)
# Long Duration (~4min)
# => Worse performance than KNN and tree-based models (notably, orig better than preprocessed)


### Tree-Based Models ###
hgb_default = HistGradientBoostingRegressor(random_state=rs, loss='squared_error')
hgb_default = TransformedTargetRegressor(
    regressor=hgb_default,
    func=np.log1p,
    inverse_func=np.expm1
)
hgb_pipe_orig = create_model_pipe(preprocessor_orig, hgb_default)
hgb_pipe_adjusted = create_model_pipe(preprocessor_pipe, hgb_default)
# Long Duration (~1mins)


rf_default = RandomForestRegressor(random_state=rs, n_jobs=-1, criterion='squared_error')
rf_default = TransformedTargetRegressor(
    regressor=rf_default,
    func=np.log1p,
    inverse_func=np.expm1
)
rf_pipe_orig = create_model_pipe(preprocessor_orig, rf_default)
rf_pipe_adjusted = create_model_pipe(preprocessor_pipe, rf_default)
# Long Duration (~5mins)
# Good performance -> further hyperparameter tuning

et_default = ExtraTreesRegressor(random_state=rs, n_jobs=-1, criterion='squared_error')
et_default = TransformedTargetRegressor(
    regressor=et_default,
    func=np.log1p,
    inverse_func=np.expm1
)
et_pipe_orig = create_model_pipe(preprocessor_orig, et_default)
et_pipe_adjusted = create_model_pipe(preprocessor_pipe, et_default)
# Long Duration (~7mins)
# Good performance -> further hyperparameter tuning

### Kernel-Based Models ###
# SVR was tried but performed bad and then removed due to long running time
svr_default = SVR()
svr_default = TransformedTargetRegressor(
    regressor=svr_default,
    func=np.log1p,
    inverse_func=np.expm1
)
svr_pipe_orig = create_model_pipe(preprocessor_orig, svr_default)
svr_pipe_adjusted = create_model_pipe(preprocessor_pipe, svr_default)
# Long Duration (~12mins)
# => Much worse performance than other models -> not further optimized

##### 3.3.2) Run the models

In [ ]:
default_models_orig = {
    "Baseline_Median_orig": baseline_median_pipe_orig,
    # "LinearReg_orig": linear_reg_pipe_orig,
    # "KNN_orig": knn_pipe_orig,
    # "MLP_orig": mlp_pipe_orig,
    # "HGB_orig": hgb_pipe_orig,
    # "RF_orig": rf_pipe_orig,
    # "ET_orig": et_pipe_orig,
    # "SVR_orig": svr_pipe_orig, # SVR was tried but performed bad and then removed here due to long running time
}

default_orig_models_results_df = get_cv_results(default_models_orig, X_train, y_train, cv=cv, rs=rs)
display(default_orig_models_results_df)

# Long Duration (~10mins)

In [ ]:
default_models = {
    # "LinearReg": linear_reg_pipe_adjusted,
    # "KNN": knn_pipe_adjusted,
    # "MLP": mlp_pipe_adjusted,
    # "HGB": hgb_pipe_adjusted,
    "RF": rf_pipe_adjusted,
    "ET": et_pipe_adjusted,
    # "SVR": svr_pipe_adjusted, # SVR was tried but performed bad and then removed here due to long running time
}

default_models_results_df = get_cv_results(default_models, X_train, y_train, cv=cv, rs=rs)
display(default_models_results_df)

# Long Duration (~5mins)

##### 3.3.3) Baseline Model Comparison Discussion of Results

In [ ]:
merged_model_results_df = pd.concat([default_orig_models_results_df, default_models_results_df])
merged_model_results_df = merged_model_results_df.sort_values(["preprocessing", "val_MAE"])
display(merged_model_results_df)

**1) Comparative performance under optimized preprocessing**

Under the optimized preprocessing pipeline, the ranking is dominated by **tree-based ensembles**:
 
- **ExtraTrees (ET, optimized)** is best overall with **val MAE = 1234.50 ± 10.37**, **val RMSE = 2129.62**, **val R² = 0.9522**.  

- **RandomForest (RF, optimized)** is a close second (**val MAE = 1236.17 ± 12.19**, **val RMSE = 2170.35**, **val R² = 0.9504**).
 
Both outperform the next tier (HGB and KNN), and they do so with very small fold-to-fold variability, indicating that the gain is not driven by a subset of “lucky” splits.
 
The remaining models behave as expected for this task:

- **HGB/KNN** improve with preprocessing but remain worse than ET/RF, consistent with residual bias (HGB) or limited representational flexibility (KNN).

- **Linear Regression** underperforms substantially, consistent with a pricing function that is not well-approximated by a global linear mapping in the engineered feature space.

- **MLP (optimized)** is unstable: despite a moderate MAE, the **RMSE explodes** and **R² becomes strongly negative**, which is characteristic of rare but extremely large errors (tail instability), making it unsuitable in this configuration.
 
---
 
**2) Effect size of preprocessing (optimized vs original)**

The optimized preprocessing delivers a large, consistent improvement for the models that can exploit nonlinear interactions:
 
- **ET:** val MAE improves from **1442.82 → 1234.50** (Δ ≈ -208; ≈ **-14.4%**) and val R² increases from **0.9305 → 0.9522**.  

- **RF:** val MAE improves from **1464.05 → 1236.17** (Δ ≈ -228; ≈ **-15.6%**) and val R² increases from **0.9276 → 0.9504**.  

- **HGB/KNN** show similarly strong absolute MAE reductions, but remain below ET/RF in final validation performance.
 
The baseline median predictor (MAE ≈ 6801, negative R²) provides a clear reference: the optimized ET/RF pipelines are not incremental improvements but a substantial step-change in predictive quality.
 
An important nuance is that **Linear Regression becomes worse** under optimized preprocessing (MAE ≈ 2242 vs ≈ 1977 originally). This is consistent with preprocessing/feature engineering increasing nonlinear structure and interactions that benefit trees but do not translate into a better linear fit.
 
---
 
**3) Detailed focus: why ExtraTrees is the strongest model here**

**ExtraTrees (optimized)** is the strongest model by a small but consistent margin over RF, and the pattern of metrics supports that this is a real advantage rather than noise:
 
- **Best mean performance** (lowest MAE and RMSE; highest R² among all candidates under the same preprocessing).

- **Highest stability** across folds (lowest std(MAE) among the strong models).
 
From a modelling perspective, ET’s advantage is plausible for this dataset: it aggregates many randomized decision trees, which can (i) capture complex non-linearities and heterogeneous feature interactions, (ii) naturally model threshold effects common in pricing (e.g., year, mileage, engine size, transmission interactions), and (iii) reduce variance via ensemble averaging. In other words, ET is well-matched to structured tabular data with mixed numeric/categorical signals and non-additive effects.
 
---
 
**4) Generalization and overfitting signals (train vs validation)**

Train–validation gaps provide an additional diagnostic:
 
- **ET/RF optimized** show materially lower training error than validation error (ET train MAE ≈ 446 vs val MAE ≈ 1234; RF train MAE ≈ 674 vs val MAE ≈ 1236). This indicates high capacity and some degree of overfitting, which is expected for ensembles. Crucially, this does **not** translate into unstable validation results: fold variability is low, suggesting the models generalize reliably despite the gap.
 
- **HGB optimized** has relatively close train/val errors (train MAE ≈ 1359 vs val MAE ≈ 1406), consistent with a more bias-limited model: it generalizes more “smoothly” but cannot reach the same accuracy ceiling as ET/RF.
 
- **MLP optimized** exhibits a clear robustness failure: the huge RMSE and negative R² indicate that a minority of cases are predicted extremely poorly. This is not merely “worse average performance”; it is a qualitatively different error profile (high-risk tail behaviour).
 
A notable anomaly appears for **ET_orig**: the reported **train MAE ≈ 4** and **train RMSE ≈ 89** are extraordinarily small compared to all other models and configurations. Given that validation performance is far from perfect, such near-zero training error suggests either extreme memorization and/or an artefact in how training metrics were computed in that configuration. In contrast, ET under optimized preprocessing yields plausible training metrics while also improving validation performance, which strengthens the credibility of the optimized pipeline as the final modelling choice.
 
---
 
**5) Bottom-line conclusion from the evidence**

The results support a clear empirical conclusion:
 
- With optimized preprocessing, **ExtraTrees is the best-performing and most stable model** on validation metrics (MAE/RMSE/R²), with RandomForest as a very close runner-up.

- The optimized preprocessing is not cosmetic: it delivers **double-digit percentage improvements** in MAE for the strongest learners.

- Models outside tree ensembles either underfit (Linear Regression), fall short on accuracy (HGB/KNN), or exhibit unacceptable tail instability (MLP optimized).
 
Within this benchmark, **ET + optimized preprocessing** is the strongest overall solution by both accuracy and stability criteria.

 

`MAE Performance` of Models on original and optimized data preparation

In [ ]:
# Create MAE_gap
merged_model_results_df["MAE_gap"] = merged_model_results_df["val_MAE"] - merged_model_results_df["train_MAE"]
merged_model_results_df["base_model"] = merged_model_results_df["model"].str.replace("_orig", "", regex=False)

# Original vs Optimized preprocessing (Validation MAE)
plt.figure(figsize=(12, 6))

for prep in ["original", "optimized"]:
    subset = merged_model_results_df[merged_model_results_df["preprocessing"] == prep]
    plt.scatter(subset["base_model"], subset["val_MAE"], label=prep)

plt.ylabel("Validation MAE")
plt.title("Original vs Optimized Preprocessing (Validation MAE)")
plt.legend()
plt.tight_layout()
plt.show()

`Train-Val-Gap` to analyze overfitting

In [ ]:
# Validation vs Train MAE (grouped bar plot)
plt.figure(figsize=(12, 6))
x = range(len(merged_model_results_df))
plt.bar(x, merged_model_results_df["train_MAE"], width=0.4, label="Train MAE")
plt.bar(
    [i + 0.4 for i in x],
    merged_model_results_df["val_MAE"],
    width=0.4,
    label="Validation MAE"
)

plt.xticks([i + 0.2 for i in x], merged_model_results_df["model"], rotation=45, ha="right")
plt.ylabel("MAE")
plt.title("Train vs Validation MAE by Model")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Generalization gap (Val − Train MAE)
plt.figure(figsize=(12, 6))

plt.bar(merged_model_results_df["model"], merged_model_results_df["MAE_gap"])
plt.axhline(0, linestyle="--")

plt.ylabel("MAE Gap (Validation − Train)")
plt.title("Generalization Gap by Model")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

#### 3.4) Optimization: Hyperparameter Tuning of Top Candidates

**Tune Top candidates with Randomized Search CV:**     

After the first runs we only keep the **top candidates for further hyperparameter** tuning to focus on most promising approaches and not waste computing power.     
We tune using [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) which calls the pipeline object for consistent preprocessing. An example by sklearn of calling the pipeline similar to this can be found [here](https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html#sphx-glr-auto-examples-compose-plot-compare-reduction-py).
Within the process of tuning, the parameters were iteratively optimized to decrease size of the search space of the top candidates. This allowed faster runtimes and better final results compared to running one extensive search on a big hyperparameter grid.

We use `squared_error` to find hyperparameters but use `absolute_error` only to optimize for MAE (primary metric) for final retraining on the best Model because it is [significantly slower](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) (~5x)

To visualize the performance of different hyperparameters, we plot the 2 arguable most impactful parameters to be tuned and their achieved val MAE. When analyzing the plot, one has to be aware that the performance is also influenced by features that are not included in the plot so one should focus only rough directions instead of relying on exact performances.

##### 3.4.1) [Tree-Based] RandomForest

In [ ]:
# Old parameter distribution
rf_param_dist = {
    "model__regressor__criterion": ["squared_error"],       # optimizes way faster than "absolute_error"
    "model__regressor__n_estimators": randint(300, 350),    # number of trees
    "model__regressor__max_depth": randint(18, 22),         # depth of each tree
    "model__regressor__min_samples_split": randint(4, 6),   # min samples to split an internal node
    "model__regressor__min_samples_leaf": randint(1, 3),    # min samples per leaf (increse to not overfit)
    "model__regressor__max_features": ["sqrt"],             # feature sampling strategy (sqrt performed better than log2 and None in previous tests)
    # "model__regressor__bootstrap": [True],                  # True is default for RFs
}

# So far best parameter distribution based on previous runs to focus search space
# rf_param_dist = {
#     "preprocess__fs__vt__threshold": [0.0],
#     "model__regressor__criterion": ['squared_error'] # For final optmization 'absolute_error'
#     "model__regressor__n_estimators": [341],
#     "model__regressor__max_depth": [20],
#     "model__regressor__min_samples_split": [4],
#     "model__regressor__min_samples_leaf": [1],
#     "model__regressor__max_features": ["sqrt"],
#     "model__regressor__oob_score": [True],
# }

rf_tuned_pipe, rf_random_search_object, rf_scores_dict = model_hyperparameter_tuning(X_train,
                                                                                     y_train,
                                                                                     cv,
                                                                                     rf_pipe_adjusted,
                                                                                     rf_param_dist,
                                                                                     n_iter=1,
                                                                                     verbose_features=[["model__regressor__n_estimators", "model__regressor__max_depth"],],
                                                                                     verbose_metric="mae",
                                                                                     verbose_plot=True,
                                                                                     verbose_top_n=20)

# Long Duration (~1min with squared_error, ~6min with absolute_error)
# Long Duration (30mins with 100 fits)

##### 3.4.2) [Tree-Based] Extra Trees

In [ ]:
# Old parameter distribution
et_param_dist = {
    "model__regressor__criterion": ["squared_error"],           # optimizes way faster than "absolute_error"
    "model__regressor__n_estimators": randint(350, 400),        # number of trees
    "model__regressor__max_depth": randint(23, 25),             # depth of each tree
    "model__regressor__min_samples_split": randint(6, 8),       # min samples to split an internal node
    "model__regressor__min_samples_leaf": randint(1, 3),        # min samples per leaf
    "model__regressor__max_features": [0.8, 0.9],            # feature sampling strategy
    # "model__regressor__bootstrap": [False]                      # default for ETs
}

et_tuned_pipe_squared_err, et_random_search_object, et_scores_dict = model_hyperparameter_tuning(X_train, y_train, cv,
                                                                                    et_pipe_adjusted,
                                                                                    et_param_dist,
                                                                                    n_iter=1,
                                                                                    verbose_features=[["model__regressor__n_estimators", "model__regressor__max_depth"],],
                                                                                    verbose_metric="mae",
                                                                                    verbose_plot=True,
                                                                                    verbose_top_n=20)

# Save here for deployment (see Section 7) because refit on entire train data already done inside model_hyperparameter_tuning
joblib.dump(et_tuned_pipe_squared_err, "et_tuned_pipe_squared_err.pkl")

# Long Duration (~20min with "absolute_error" on n_iter = 1)

# Use "absolute_error" for final best performance
# MAE: 1187.1061
# RMSE: 2104.8590
# R²: 0.9533
# Best Model params: {'model__regressor__n_estimators': 395, 'model__regressor__min_samples_split': 7, 'model__regressor__min_samples_leaf': 1, 'model__regressor__max_features': 0.8, 'model__regressor__max_depth': 24, 'model__regressor__criterion': 'absolute_error'}

##### 3.4.3) Comparison of Tuned Models

In [ ]:
# Use object from randomizedsearch to retrieve the mean metrics of the best model (that was also refit on entire data for final predictions later)
model_scores = {"rf_tuned": rf_scores_dict, "et_tuned": et_scores_dict}

# Convert dictionary to DataFrame (transpose to have models as rows) and sort by val_mae (primary metric)
df_scores = pd.DataFrame(model_scores).T 
df_scores = df_scores[['val_mae', 'val_rmse', 'val_r2','train_mae', 'train_rmse', 'train_r2']]
df_scores = df_scores.sort_values(by='val_mae')

print("Model Comparison Table:")
display(df_scores)

 **Hyperparameter changes (default → tuned) and their expected effects**
 
- **RandomForest (tuned)**

  - `n_estimators=341` (↑): lowers estimator variance via stronger averaging.

  - `max_depth=20` (None → capped): limits tree complexity and reduces overfitting.

  - `min_samples_split=4` (2 → 4): prevents splits supported by very few samples; smoother trees.

  - `min_samples_leaf=1` (unchanged): allows fine local structure.

  - `max_features="sqrt"` (1.0 → sqrt): increases tree diversity; typically improves generalization.

  - `criterion="squared_error"` (unchanged): efficient optimization; not directly aligned with MAE.

  - Pipeline/FS: `preprocess__fs__vt__threshold=0.0` (constant-feature removal only).
 
- **ExtraTrees (tuned)**

  - `criterion="absolute_error"` ("squared_error" → "absolute_error"): shifts optimization toward MAE/median-type robustness, reducing sensitivity to outliers; typically slower.

  - `n_estimators=395` (↑): variance reduction through averaging.

  - `max_depth=24` (None → capped): complexity control while allowing deep interactions.

  - `min_samples_split=7` (2 → 7): stronger regularization than RF; discourages overly specific splits.

  - `min_samples_leaf=1` (unchanged): retains capacity for local patterns.

  - `max_features=0.8` (1.0 → 0.8): increases diversity across trees; supports generalization.
 
Interpretation of the tuning choices:

- Both tunings follow standard bias–variance control (more trees, depth caps, split constraints, feature subsampling).

- The ET configuration additionally changes the loss from MSE-type to MAE-type (`absolute_error`), which is directly aligned with the primary evaluation metric.
 
---
 
**Observed tuned performance (mean CV)**
 
- On validation, ET is better on **MAE** (primary) and also improves **RMSE** and **R²**. The MAE difference is small (~4), but the direction is consistent across all three metrics.

- Both models show substantial train–validation gaps, indicating overfitting in absolute terms; ET fits the training folds more strongly, yet still yields the best validation metrics, consistent with high-capacity ensembles combined with strong randomization and averaging.
 
---
 
**What can be concluded (and what cannot)**
 
- Supported by the reported results: `et_tuned_pipe` is the best tuned candidate under the chosen objective (minimize validation MAE), with corroborating improvements in RMSE and R².

- Not supported without additional evidence: the magnitude of improvement relative to the default configurations (default CV metrics are not provided here), and statistical significance of the ET–RF differences (would require fold-wise distributions and paired testing).
 
---
 
**Decision**
 
- Final choice: **use `et_tuned_pipe`**, because it attains the lowest validation MAE and simultaneously improves RMSE and R² under identical CV splits, and because its tuned objective (`absolute_error`) is explicitly aligned with MAE-focused evaluation.

 

In [ ]:
plot_train_val_comparison(df_scores)

##### 3.4.4) Optimize Best Model further on absolute error (primary metric)

Our best model is now trained optimizing the `absolut_error` with the best hyperparameters found when optimizing the `squared_error`. This approach is taken because optimizing the absolut_error is computationally too expensive and therefore only used for the final training.

In [ ]:
# Use best parameter distribution and run it with "absolute_error" to minimize MAE even further
# TODO remove old
# et_param_dist = {
#     "model__regressor__criterion": ["absolute_error"], # absolute_error for final best prediction
#     "model__regressor__n_estimators": [395],
#     "model__regressor__max_depth": [24],
#     "model__regressor__min_samples_split": [7],
#     "model__regressor__min_samples_leaf": [1],
#     "model__regressor__max_features": [0.8],
# }
# => MAE: 1183.6165

et_param_dist = {
    "model__regressor__criterion": ["absolute_error"], # absolute_error for final best prediction
    "model__regressor__n_estimators": [385],
    "model__regressor__max_depth": [23],
    "model__regressor__min_samples_split": [7],
    "model__regressor__min_samples_leaf": [1],
    "model__regressor__max_features": [0.8],
}
# => MAE: 1188.1039

# We use the same function to get a final result CV result with "absolut_error" and then the model is automatically refit on all data
et_tuned_pipe, et_random_search_object, et_scores_dict = model_hyperparameter_tuning(X_train, y_train, cv, 
                                                                                     et_pipe_adjusted,
                                                                                     et_param_dist,
                                                                                     n_iter=1,
                                                                                     verbose_features=[],
                                                                                     verbose_plot=False)

joblib.dump(et_tuned_pipe, "et_tuned_pipe.pkl")

#### 3.5) Visualizations and Insights of the entire Pipeline

**Preprocessing is independet of model:**     
To ensure a consistent comparison, the preprocessing pipeline is independent of the model. Therefore, the insights are the same for all models.


**Visualize outputs** of each preprocessing step:     
We use this `DebugTransformer` to print the data shape and check analysis of y-data-profiling for missing values etc. in steps where we would not expect them. This facilitates the experimenting process massively and helps finding errors within the pipeline fast.     
For the visualizations we use ydata-profiling instead of our own plots like in the EDA for a more concise output.


In [ ]:
# Toggle verbosity of output by setting show_data or even y_data_profiling to True
show_data = True
y_data_profiling = False

# Set output to pandas DataFrames for easier inspection while we use numpy arrays for efficient model training (default)
enc_transf_scale.set_output(transform="pandas")
fs_pipe.set_output(transform="pandas")

##### 3.5.1) Start

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ('debug_start', DebugTransformer('START', show_data=show_data, y_data_profiling=y_data_profiling)),
])

print("Show outputs of each step in the preprocessing pipeline:")

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.2) Cleaning

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True, verbose=True)),
    ('debug_after_clean', DebugTransformer('AFTER CLEANING', show_data=show_data, y_data_profiling=y_data_profiling)),
])

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.3) [Unused] Outlier Handling

In [ ]:
# The code for debugging and visualizing the outlier handling is included in the unused_experiments.ipynb

##### 3.5.4) Missing Values Handling

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    # ("outliers", OutlierHandler(
    #     cols=[c for c in orig_numeric_features if c != "mileage"],      # only original numeric features here, no mileage because of log transform later
    #     methods=("iqr", "mod_z"),                                       # robust voting
    #     min_votes=2,                                                    # outlier if both methods agree
    #     iqr_k=1.5,
    #     z_thresh=3.5,
    #     action="clip",                                                   
    #     verbose=False,
    # )),
    ("imputer", IndividualHierarchyImputer()),
    ('debug_after_impute', DebugTransformer('AFTER IMPUTATION', show_data=show_data, y_data_profiling=y_data_profiling)),
])

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.5) Feature Engineering

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020, verbose=True)),
    ('debug_after_fe', DebugTransformer('AFTER FEATURE ENGINEERING', show_data=show_data, y_data_profiling=y_data_profiling)),
])

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.6) Transformation, Scaling, Encoding

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ("ct", (enc_transf_scale)),
    ('debug_after_ct', DebugTransformer('AFTER COLUMN TRANSFORMER', show_data=show_data, y_data_profiling=y_data_profiling))
])

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.7) Feature Selection

In [ ]:
debug_preprocessor_pipe = Pipeline([
    ("clean", CarDataCleaner(handle_electric="other", set_carid_index=False, use_fuzzy=True)),
    # [Unused Outlier Handling]
    ("imputer", IndividualHierarchyImputer()),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ("ct", (enc_transf_scale)),
    ("fs", (fs_pipe)),
    ('debug_after_fs', DebugTransformer('AFTER FEATURE SELECTION', show_data=show_data, y_data_profiling=y_data_profiling))
])

# We call fit_tranform here on the entire training data to just visualize the result. The insights from here are not used for anything else in model decisions so it's not leakage
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

##### 3.5.8) Visualize entire pipe before adding the model

In [ ]:
debug_preprocessor_pipe

##### 3.5.9) Model

For a detailed Inspection of the best model refer to the Feature Importance analysis in the open-end-section.

In [ ]:
# Use the debug preprocessor pipeline to get final feature names by hierarchically accessing each step
feature_names_after_fs = debug_preprocessor_pipe.named_steps['fs'].get_feature_names_out()
feat_names = feature_names_after_fs
importances = et_tuned_pipe.named_steps["model"].regressor_.feature_importances_
df_feat_importance_et = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
 
print("Feature Importances:")
for _, row in df_feat_importance_et.iterrows():
    print(f"{row['feature']:30s}: {row['importance']:.6f}")

### IV. Open-Ended-Section

#### 4.1 SHAP Interpretability for Our Final Tree Model (Informative Only)

**a) Objective and motivation**

After our end-to-end pipeline is finished, we use **SHAP (SHapley Additive exPlanations)**.

Goals:
- Identify the **most influential features** for the final tuned model (`et_tuned_pipe`).
- Validate whether feature effects are **plausible** (age, mileage, engine, etc.).
- Check how much **target encodings** and engineered interactions contribute.

Important: **SHAP does not change the model or feature set.** We do not build a new pipeline based on SHAP.

---

**b) Difficulty of the task**

This is non-trivial because SHAP must explain the model input **after** our preprocessing and feature selection:

- The model does not see raw columns. 
It sees: engineered numeric features (e.g., interactions, relative features, logs), OHE columns, target-encoded columns and the reduced subset after **VT + majority voting**.
- We therefore reconstruct:
  - the exact **post-preprocess feature matrix**, and
  - aligned **feature names** after applying both selection masks (VT support + majority selector mask).
- Because the full pipeline includes engineered preprocessing + selection, we treat the tuned pipeline as a **black box** and use SHAP via a **PermutationExplainer** (robust but expensive).
- Runtime: SHAP is costly, so we explain only a **subsample** (`sample_size=1000`) with a small background set.

---

**c) Correctness and efficiency**

We kept the analysis correct and consistent with the production pipeline:

- **No leakage / no optimization loop:** SHAP is computed on the already-fitted `et_tuned_pipe` and used only for interpretation.
- **Exact alignment:** feature names come from the ColumnTransformer output and are then filtered by VT + majority voting masks.
- **Global SHAP importance:** features are ranked by mean absolute contribution:
  
  $$
  Importance(feature_j) = \frac{1}{N}\sum_{i=1}^{N} |SHAP_{i,j}|
  $$

- **Efficient computation:** stable ranking via subsampling (PermutationExplainer on 1000 rows; runtime ~21 minutes in our run).

---

**d) Results and interpretation**

Model context:
- Final tuned model: `et_tuned_pipe` (**ExtraTrees**)
- Total features used after preprocessing + FS: **28**
- SHAP explainer used: **PermutationExplainer** (1001 iterations; ~21:34 total)

Top drivers (mean |SHAP|), excerpt:

| Feature | Importance | Interpretation |
|---|---:|---|
| `mean_te__model` | 1486.36 | Model-level mean target encoding (strong market-value proxy) |
| `median_te__model` | 1409.38 | Model-level median target encoding (strong market-value proxy) |
| `num__mpg_x_age` | 870.81 | Interaction capturing “efficiency x age” effects |
| `cat__transmission_Manual` | 846.46 | Manual transmission effect (dataset-dependent) |
| `num__age` | 764.79 | Direct age penalty / depreciation signal |
| `num__engineSize` | 655.15 | Engine size (segment/performance proxy) |
| `num__age_rel_brand` | 628.79 | Age relative to typical age inside the brand |
| `log__mileage` | 592.53 | Non-linear mileage effect (diminishing marginal impact) |
| `median_te__brand_trans` | 521.83 | Brand x transmission median target encoding |
| `num__age_rel_model` | 488.77 | Age relative to typical age within the model |
| `num__engine_per_mpg` | 303.93 | Performance/efficiency ratio proxy |
| `log__miles_per_year` | 223.94 | Usage intensity (mileage normalized by age) |

Key takeaways:
- **Target encodings still dominate** global importance (model-level mean/median TE). This is expected: model identity carries a large fraction of price signal.
- **Transmission became a top driver** in the ExtraTrees variant (`cat__transmission_Manual` ranks #4), suggesting stronger split usage on this categorical signal compared to the previous RF run.
- **Age and mileage remain major drivers**, and appear in intuitive forms (`num__age`, `log__mileage`, relative age features, and `log__miles_per_year`), supporting interpretability.
- **Engine/performance interactions matter** (`num__engineSize`, `num__engine_per_mpg`, `num__mpg_x_age`, `num__mpg_x_engine`), indicating feature engineering adds useful non-linear structure beyond raw variables.

Beeswarm plot (distribution of effects), main observations:
- **`mean_te__model` and `median_te__model` show the widest SHAP spread** → model identity (via target encoding) is the strongest pricing signal.
- **Manual transmission shows a clear directional pattern** in this tuned ExtraTrees model: `cat__transmission_Manual=1` tends to push predictions **down** (negative SHAP), while `=0` tends to push them **up** (positive SHAP), with heterogeneity explained by brand/model interactions.
- **Mileage is clearly non-linear** (`log__mileage`): low mileage contributes positively; high mileage pushes predictions down with diminishing marginal impact.
- **Age effects are consistent** (`num__age`, `num__age_rel_brand`, `num__age_rel_model`): being older (especially older than “typical” for brand/model) reduces predicted price.

---

**e) Alignment with objectives**

This section adds transparency without changing the modeling procedure:

- Feature selection stays **VT + majority voting** (robust and model-agnostic).
- SHAP is used **only** to explain the final tuned model (`et_tuned_pipe`).
- The resulting drivers (target encodings + age/mileage/engine + interactions + transmission) are consistent with domain logic and support trust in the final pipeline.


##### Functions

In [ ]:
# Get Feature names aligned with X_proc (after preprocess incl. VT + majority voting)
def get_pipeline_feature_matrix(pipe, X, debug_preprocessor_pipe):
    """
    Given a fitted model pipeline with steps:
      'preprocess' -> 'model'
    where preprocess itself is a Pipeline:
      clean -> group_imputer -> fe -> ct -> fs(vt + selector)
    return:
      X_proc: 2D numpy array of features just before the model step
      feat_names: 1D np.array of feature names aligned with X_proc columns
    """
    pre = pipe.named_steps["preprocess"]

    # Transform to model-ready matrix and get feature names debug preprocessor
    X_proc = pre.transform(X)
    feat_names = debug_preprocessor_pipe.named_steps['fs'].get_feature_names_out()

    return X_proc, feat_names


In [ ]:
# Compute SHAP Importance
def compute_shap_importance(
    pipe,
    X,
    sample_size=1000,
    seed=rs,
    model_name=None,
):
    """
    Compute global SHAP feature importances for a fitted pipeline (informative only).

    Fix:
      - TreeExplainer additivity check can fail for some sklearn tree implementations (incl. HGB).
        We disable it via check_additivity=False.
      - If TreeExplainer still fails, fall back to a model-agnostic SHAP explainer.
    """
    # Extract processed feature matrix and names
    X_proc, feat_names = get_pipeline_feature_matrix(pipe, X, debug_preprocessor_pipe)

    # Subsample rows for SHAP (for speed)
    rng = np.random.default_rng(seed)
    n = min(sample_size, len(X_proc))
    idx = rng.choice(len(X_proc), n, replace=False)
    X_sample = X_proc[idx]

    # Underlying model (last step in pipeline)
    model = pipe.named_steps["model"]
    tag = model_name or model.__class__.__name__

    # Background for SHAP (small subset)
    bg_n = min(200, len(X_sample))
    bg_idx = rng.choice(len(X_sample), bg_n, replace=False)
    X_bg = X_sample[bg_idx]

    # Try TreeExplainer first (fast for tree models)
    try:
        explainer = shap.TreeExplainer(model, X_bg)
        shap_vals = explainer.shap_values(X_sample, check_additivity=False)

        # shap_vals can be list-like in some setups; regression should be 2D
        if isinstance(shap_vals, list):
            shap_vals = shap_vals[0]

        base_vals = getattr(explainer, "expected_value", 0.0)
        shap_values = shap.Explanation(
            values=shap_vals,
            base_values=np.full((len(X_sample),), base_vals) if np.isscalar(base_vals) else base_vals,
            data=X_sample,
            feature_names=feat_names,
        )

    except Exception as e:
        # Fallback: model-agnostic explainer (slower but robust)
        explainer = shap.Explainer(model.predict, X_bg, feature_names=feat_names)
        shap_values = explainer(X_sample)

    importance = np.abs(shap_values.values).mean(axis=0)

    shap_df = (
        pd.DataFrame({"feature": feat_names, "importance": importance})
        .sort_values("importance", ascending=False)
        .reset_index(drop=True)
    )

    print(f"Features by SHAP for {tag}:")
    print(shap_df.head(40).to_string(index=False))

    return shap_df, feat_names, shap_values, X_sample


In [ ]:
# SHAP Plots
def plot_top_shap_bar(shap_df, model_name, top_k):
    """
    Horizontal bar plot of top_k features by mean |SHAP|.
    """
    top_df = shap_df.head(top_k).iloc[::-1]  # reverse for nicer barh order
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.barh(top_df["feature"], top_df["importance"])
    ax.set_xlabel("Average |SHAP| value")
    ax.set_title(f"Top {top_k} features by SHAP – {model_name}")
    plt.tight_layout()
    plt.show()


def plot_shap_beeswarm(shap_values, X_sample, feat_names, model_name, max_display=20):
    """
    SHAP summary (beeswarm) plot for top features.
    """
    X_df = pd.DataFrame(X_sample, columns=feat_names)

    # Create one figure and tell SHAP not to auto-show
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values.values, X_df, max_display=max_display, show=False)

    plt.title(f"SHAP Beeswarm – {model_name}")
    plt.tight_layout()
    plt.show()


##### SHAP of Best Model

In [ ]:
# ExtraTrees baseline report + SHAP
et_pipe = et_tuned_pipe_squared_err # Use pipe tuned on squared error here again for computational reasons

# Feature matrix + names after preprocess (clean+impute+fe+ct+fs)
X_proc_et, feat_names_et = get_pipeline_feature_matrix(et_pipe, X_train, debug_preprocessor_pipe)
n_features_total_et = X_proc_et.shape[1]

print("ExtraTrees (tuned pipe) - feature space info:")
print(f"Total features used: {n_features_total_et}")

shap_importance_et, feat_names_et, shap_vals_et, X_sample_et = compute_shap_importance(
    et_pipe,
    X_train,
    sample_size=1000,
    seed=rs,
    model_name="ExtraTrees",
)

plot_top_shap_bar(shap_importance_et, model_name="ExtraTrees", top_k=n_features_total_et)
plot_shap_beeswarm(shap_vals_et, X_sample_et, feat_names_et, model_name="ExtraTrees", max_display=n_features_total_et)

#### 4.2 Global vs Brand- and Model-Specific Models

**a) Objective and motivation**

We investigated how far Cars4You should specialize its pricing models:

1. **Brand level:** Is a single global price model sufficient, or do brand-specific models reduce pricing error?
2. **Brand–model level:** For frequent (brand, model) segments (e.g. “VW Golf”, “Skoda Octavia”), does an even more specialized model per segment bring additional improvements, or does it overfit?

Starting point is our final tuned production pipeline **`et_tuned_pipe`** (full preprocessing + tuned **ExtraTrees** regressor). We compare:

- **Global model:** trained on all cars, evaluated only on a given segment.
- **Brand-specific model:** same pipeline structure and hyperparameters, fitted only on cars of a given brand.
- **Brand–model-specific model:** same pipeline structure and hyperparameters, fitted only on cars of a given (brand, model) pair.

We measured **MAE** and **RMSE** per segment using **5-fold cross-validation**. This quantifies the gain/loss when moving from:

> one global model → several brand models → many brand–model models.

---

**b) Difficulty of the tasks**

This multi-level comparison is not easy because it requires leakage-free, segment-wise evaluation inside cross-validation:

- **Per-segment metrics inside CV (not a single score):** each fold must report MAE/RMSE *for specific brands/pairs* on the fold’s validation set.
- **Fair protocol for global vs specialized models (within each fold):**
  - global model is trained on all training rows, but evaluated only on validation rows belonging to the segment;
  - segment-specific model is trained and evaluated only on that segment’s rows.
- **Imbalanced / small segments:** data is heavily skewed across brands/models, so we enforce minimum segment sizes:
  - **brand level:** only brands with **≥ 500** samples overall;
  - **brand–model level:** only pairs with **≥ 80** samples overall;
  - plus per-fold minimum training-size checks to avoid unstable tiny fits.
- **Cleaning labels before grouping:** inconsistent text labels (casing/spacing/typos) can split real segments; we normalize `(brand, model)` once using the same cleaning logic as in the pipeline and use cleaned labels for masks.
- **Manual RMSE:** computed as `sqrt(MSE)` inside the CV loops for compatibility with our environment.

---

**c) Correctness and efficiency of implementation**

We kept the evaluation correct and efficient:

- **Leakage-free out-of-fold evaluation:** in each fold the pipeline is fitted only on that fold’s training rows; metrics are computed only on validation rows.
- **Single CV design reused everywhere:** identical KFold splits (`n_splits=5`, `shuffle=True`, fixed `random_state`) are reused for all comparisons, making deltas directly comparable.
- **Efficient global baseline:** the global model is fitted **once per fold**, then reused to compute metrics for many brands/pairs in that fold (instead of refitting per segment).
- **Segmentation labels separated from training data:** model training always uses the original fold rows; segment membership uses cleaned labels for correct grouping.
- **Guards for tiny segments:** segments/folds with insufficient training samples are skipped to avoid unstable conclusions.

---

**d) Discussion of results**

- **Candidate brands:** all brands
- **Candidate (brand, model) pairs:** ≥ 80 samples; 100 pairs


**Brand-level comparison (global vs brand-specific, using `et_tuned_pipe`)**

Across the main brands, brand-specific training is **not consistently beneficial**. Even when some brands can improve, the typical trade-off remains:

- **Potential benefit:** brand-specific models can capture brand-local price structure if it differs materially from the full population.
- **Common downside:** less data and reduced diversity often outweigh specialization; the global model already captures brand effects via features (including encodings), so restricting to one brand can hurt generalization.
- **Practical interpretation:** brand-level specialization should be treated as **optional** and justified only when it shows **stable negative ΔMAE/ΔRMSE** in CV.

*(Brand-level table is omitted here because the provided updated results focus on candidate selection and pair-level outcomes; the recommendation below reflects the updated ExtraTrees pipeline behavior and the observed specialization risk patterns.)*


**Brand–model comparison (global vs brand–model-specific, `et_tuned_pipe`)**

Results are clearly **mixed** even after filtering to pairs with ≥ 80 samples: some segments improve, while others degrade substantially.

**Top improvements (largest negative ΔMAE; specialized better than global):**

| (brand, model) | n | MAE_global | MAE_pair | ΔMAE | RMSE_global | RMSE_pair | ΔRMSE |
|---|---:|---:|---:|---:|---:|---:|---:|
| Audi q7 | 268 | 3186.8 | 3044.2 | **-142.6** | 4282.0 | 4110.4 | **-171.6** |
| Audi tt | 222 | 1852.2 | 1723.0 | **-129.3** | 2597.2 | 2368.0 | **-229.2** |
| Ford edge | 137 | 1106.1 | 1030.3 | **-75.7** | 1556.6 | 1420.7 | **-135.9** |
| Hyundai ioniq | 203 | 1491.9 | 1429.6 | **-62.3** | 1921.5 | 1883.2 | **-38.2** |
| Ford s-max | 201 | 1278.5 | 1242.4 | **-36.1** | 1684.1 | 1656.7 | **-27.3** |
| VW sharan | 180 | 1584.9 | 1563.3 | **-21.6** | 2112.6 | 2099.1 | **-13.6** |
| Mercedes c class | 5288 | 1904.9 | 1885.7 | **-19.3** | 2813.0 | 2867.8 | **+54.8** |
| Skoda octavia | 1021 | 944.5 | 925.4 | **-19.1** | 1333.9 | 1307.2 | **-26.6** |

Notes:
- Several segments show **consistent wins** (negative ΔMAE and negative ΔRMSE), e.g. Audi q7/tt, Ford edge, Skoda octavia.
- Some show a **MAE improvement but RMSE worsening** (e.g. Mercedes c class: ΔMAE < 0 but ΔRMSE > 0), which suggests fewer average errors but worse tail behavior/outliers.

**Top degradations (largest positive ΔMAE; specialized worse than global):**

| (brand, model) | n | MAE_global | MAE_pair | ΔMAE | RMSE_global | RMSE_pair | ΔRMSE |
|---|---:|---:|---:|---:|---:|---:|---:|
| Audi a_unknown | 94 | 2099.5 | 2705.5 | **+606.1** | 2832.7 | 3498.3 | **+665.7** |
| Mercedes s class | 134 | 3986.0 | 4469.3 | **+483.3** | 6122.6 | 6485.4 | **+362.8** |
| Mercedes cls class | 160 | 1791.0 | 2175.7 | **+384.7** | 3246.5 | 3983.2 | **+736.7** |
| VW amarok | 83 | 2326.4 | 2598.5 | **+272.1** | 3065.4 | 3424.4 | **+359.0** |
| Mercedes gle class | 327 | 2059.5 | 2259.4 | **+199.9** | 3142.4 | 3419.7 | **+277.3** |
| VW touareg | 244 | 2193.4 | 2367.2 | **+173.8** | 3393.5 | 3820.3 | **+426.8** |
| Audi q5 | 594 | 1736.5 | 1903.2 | **+166.8** | 2427.5 | 2781.8 | **+354.3** |
| BMW x5 | 313 | 2314.0 | 2476.0 | **+162.0** | 3074.4 | 3489.7 | **+415.3** |

Interpretation:
- Even with ≥ 80 samples, **pair-level specialization can overfit** and lose beneficial cross-segment learning.
- Degradations are often large in both MAE and RMSE, indicating not just noise but materially worse generalization for some pairs.

---

**e) Alignment with objectives**

This study directly answers whether additional specialization layers are justified for deployment under real data constraints, using the final tuned pipeline `et_tuned_pipe` and a consistent leakage-free CV protocol.

**Deployment recommendation (based on ExtraTrees results and the observed Δ patterns):**
- Keep a **single global model** as the default (most robust across segments).
- Consider **brand-level specialization only selectively**, and only where it demonstrates **repeatable negative ΔMAE and/or ΔRMSE** under the same CV protocol.
- Consider **brand–model specialization only as a gated option**:
  - only for pairs with sufficient data (≥ 80 samples + per-fold minimum training size),
  - only if the pair shows **stable negative ΔMAE and not unacceptable ΔRMSE** (watch tail risk),
  - and with monitoring, because many pairs **degrade** substantially (positive Δ).

This demonstrates not only a tuned final model, but also an evidence-based assessment of whether “more specialized” models actually improve pricing accuracy versus increasing complexity and overfitting risk.


##### 4.2.1 Load final tuned pipeline, define key columns, brand frequency and metric helpers

We use the squared error here and compare it to the performance on the squared error because of disproportionate computational expensive training with the absolute_error

In [ ]:
pipe_template = et_tuned_pipe_squared_err

# Required inputs
brand_col = "brand"
model_col = "model"

assert brand_col in X_train.columns, f"Missing column '{brand_col}' in X_train."
assert model_col in X_train.columns, f"Missing column '{model_col}' in X_train."

In [ ]:
cleaner = next(v for v in pipe_template.get_params(deep=True).values()
               if v.__class__.__name__ == "CarDataCleaner")

X_seg = X_train.copy()
tmp = cleaner.fit_transform(X_train.copy(), y_train)
X_seg[[brand_col, model_col]] = tmp[[brand_col, model_col]]

X_train = X_seg


In [ ]:
# Inspect brand frequencies
brand_counts = X_train[brand_col].value_counts()
display(brand_counts.head(15).to_frame("count"))


# Select candidate brands
#    - TOP_K: max number of brands to compare.
#    - MIN_SAMPLES: minimum number of rows per brand.
TOP_K = 8
MIN_SAMPLES = 500  # ensures enough data for stable per-brand estimates

candidate_brands = (
    brand_counts[brand_counts >= MIN_SAMPLES]
    .head(TOP_K)
    .index
    .tolist()
)

print("Candidate brands:", candidate_brands)

In [ ]:
# Cross-validation: same folds reused everywhere for fairness and reproducibility
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = list(cv.split(X_train, y_train))

def mae_rmse(y_true, y_pred):
    """Return MAE and RMSE (RMSE computed as sqrt(MSE) for sklearn-compatibility)."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    return mae, rmse

##### 4.2.2 Brand-level comparison (Global vs Brand-specific)

In [ ]:
def eval_global_by_brand(pipe_template, X, y, brand_col, brands, splits):
    """
    Global model evaluation per brand (out-of-fold):
    - Fit 1 global model per fold on ALL training rows.
    - Compute MAE/RMSE only on validation rows belonging to each brand.
    """
    rows = []

    for fold, (tr_idx, va_idx) in enumerate(splits, start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        pipe = clone(pipe_template)
        pipe.fit(X_tr, y_tr)
        y_pred = pipe.predict(X_va)

        for b in brands:
            mask = (X_va[brand_col] == b)
            n = int(mask.sum())
            if n == 0:
                continue

            mae, rmse = mae_rmse(y_va[mask], y_pred[mask])
            rows.append({"fold": fold, "brand": b, "MAE": mae, "RMSE": rmse, "n": n})

    df = pd.DataFrame(rows)
    summary = (
        df.groupby("brand")
          .apply(lambda g: pd.Series({
              "MAE_mean": g["MAE"].mean(),
              "MAE_std":  g["MAE"].std(ddof=0),
              "RMSE_mean": g["RMSE"].mean(),
              "RMSE_std":  g["RMSE"].std(ddof=0),
              "n": int(g["n"].sum()),
          }))
          .reset_index()
    )
    return summary


In [ ]:
def eval_brand_specific(pipe_template, X, y, brand_col, brands, splits, min_train_per_fold=50):
    """
    Brand-specific evaluation:
    - For each fold and brand: train the SAME pipeline structure only on that brand's training rows.
    - Evaluate only on that brand's validation rows.
    """
    rows = []

    for fold, (tr_idx, va_idx) in enumerate(splits, start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        for b in brands:
            tr_mask = (X_tr[brand_col] == b)
            va_mask = (X_va[brand_col] == b)

            n_tr = int(tr_mask.sum())
            n_va = int(va_mask.sum())

            # These checks are necessary: some folds can have very few samples for a segment.
            if n_va == 0 or n_tr < min_train_per_fold:
                continue

            pipe = clone(pipe_template)
            pipe.fit(X_tr[tr_mask], y_tr[tr_mask])
            y_pred_b = pipe.predict(X_va[va_mask])

            mae, rmse = mae_rmse(y_va[va_mask], y_pred_b)
            rows.append({"fold": fold, "brand": b, "MAE": mae, "RMSE": rmse, "n": n_va})

    df = pd.DataFrame(rows)
    summary = (
        df.groupby("brand")
          .apply(lambda g: pd.Series({
              "MAE_mean": g["MAE"].mean(),
              "MAE_std":  g["MAE"].std(ddof=0),
              "RMSE_mean": g["RMSE"].mean(),
              "RMSE_std":  g["RMSE"].std(ddof=0),
              "n": int(g["n"].sum()),
          }))
          .reset_index()
    )
    return summary


In [ ]:
df_global_brand = eval_global_by_brand(
    pipe_template, X_train, y_train, brand_col, candidate_brands, splits
).rename(columns={
    "MAE_mean": "MAE_mean_global", "MAE_std": "MAE_std_global",
    "RMSE_mean": "RMSE_mean_global", "RMSE_std": "RMSE_std_global",
    "n": "n_global"
})

df_brand_spec = eval_brand_specific(
    pipe_template, X_train, y_train, brand_col, candidate_brands, splits, min_train_per_fold=50
).rename(columns={
    "MAE_mean": "MAE_mean_brand", "MAE_std": "MAE_std_brand",
    "RMSE_mean": "RMSE_mean_brand", "RMSE_std": "RMSE_std_brand",
    "n": "n_brand"
})

df_compare_brand = df_global_brand.merge(df_brand_spec, on="brand", how="inner")
df_compare_brand["delta_MAE"] = df_compare_brand["MAE_mean_brand"] - df_compare_brand["MAE_mean_global"]
df_compare_brand["delta_RMSE"] = df_compare_brand["RMSE_mean_brand"] - df_compare_brand["RMSE_mean_global"]

df_compare_brand = df_compare_brand.sort_values("delta_MAE")
display(df_compare_brand)

# Long Duration (~3min)

In [ ]:
plt.figure(figsize=(8, 4))
x = np.arange(len(df_compare_brand))
w = 0.35

plt.bar(x - w/2, df_compare_brand["MAE_mean_global"], w, label="Global")
plt.bar(x + w/2, df_compare_brand["MAE_mean_brand"],  w, label="Brand-specific")
plt.xticks(x, df_compare_brand["brand"], rotation=45, ha="right")
plt.ylabel("MAE (GBP)")
plt.title("Global vs Brand-specific (MAE)")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 3))
plt.bar(df_compare_brand["brand"], df_compare_brand["delta_MAE"])
plt.axhline(0, linestyle="--")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Δ MAE (brand - global)")
plt.title("Effect of brand specialization (negative = MAE improvement)")
plt.tight_layout()
plt.show()


##### 4.2.3 Brand-Model comparison (Global vs Pair-specific)

In [ ]:
# Frequent (brand, model) pairs only (avoid conclusions from tiny segments)
pair_counts = (
    X_train.groupby([brand_col, model_col])
    .size()
    .sort_values(ascending=False)
)

MIN_PAIR_SAMPLES = 80  # no overfitting on low sample sizes

candidate_pairs = pair_counts[pair_counts >= MIN_PAIR_SAMPLES]

print(f"Number of candidate pairs (n >= {MIN_PAIR_SAMPLES}): {len(candidate_pairs)}")

# Show the most frequent pairs for context (readable table with names)
display(candidate_pairs.head(500).reset_index(name="count"))


In [ ]:
def eval_global_by_pair(pipe_template, X, y, brand_col, model_col, pairs, splits):
    """
    Global model evaluation per (brand, model):
    - Fit once per fold on ALL cars.
    - Score only on validation rows for each selected pair.
    """
    rows = []

    for fold, (tr_idx, va_idx) in enumerate(splits, start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        pipe = clone(pipe_template)
        pipe.fit(X_tr, y_tr)
        y_pred = pipe.predict(X_va)

        for (b, m) in pairs:
            mask = (X_va[brand_col] == b) & (X_va[model_col] == m)
            n = int(mask.sum())
            if n == 0:
                continue

            mae, rmse = mae_rmse(y_va[mask], y_pred[mask])
            rows.append({"fold": fold, "brand": b, "model": m, "MAE": mae, "RMSE": rmse, "n": n})

    df = pd.DataFrame(rows)
    summary = (
        df.groupby(["brand", "model"])
          .apply(lambda g: pd.Series({
              "MAE_mean": g["MAE"].mean(),
              "MAE_std":  g["MAE"].std(ddof=0),
              "RMSE_mean": g["RMSE"].mean(),
              "RMSE_std":  g["RMSE"].std(ddof=0),
              "n": int(g["n"].sum()),
          }))
          .reset_index()
    )
    return summary


def eval_pair_specific(pipe_template, X, y, brand_col, model_col, pairs, splits, min_train_per_fold=40):
    """
    Pair-specific models:
    - For each fold and (brand, model): fit the pipeline only on that segment's training rows.
    - Evaluate only on that segment's validation rows.

    The min_train_per_fold guard is necessary because some folds can have too few samples
    even if the pair is frequent overall.
    """
    rows = []

    for fold, (tr_idx, va_idx) in enumerate(splits, start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

        for (b, m) in pairs:
            tr_mask = (X_tr[brand_col] == b) & (X_tr[model_col] == m)
            va_mask = (X_va[brand_col] == b) & (X_va[model_col] == m)

            n_tr = int(tr_mask.sum())
            n_va = int(va_mask.sum())

            if n_va == 0 or n_tr < min_train_per_fold:
                continue

            pipe = clone(pipe_template)
            pipe.fit(X_tr[tr_mask], y_tr[tr_mask])
            y_pred = pipe.predict(X_va[va_mask])

            mae, rmse = mae_rmse(y_va[va_mask], y_pred)
            rows.append({"fold": fold, "brand": b, "model": m, "MAE": mae, "RMSE": rmse, "n": n_va})

    df = pd.DataFrame(rows)
    summary = (
        df.groupby(["brand", "model"])
          .apply(lambda g: pd.Series({
              "MAE_mean": g["MAE"].mean(),
              "MAE_std":  g["MAE"].std(ddof=0),
              "RMSE_mean": g["RMSE"].mean(),
              "RMSE_std":  g["RMSE"].std(ddof=0),
              "n": int(g["n"].sum()),
          }))
          .reset_index()
    )
    return summary

In [ ]:
pairs_list = list(candidate_pairs.index)  # list of (brand, model) tuples

df_global_pair = eval_global_by_pair(
    pipe_template, X_train, y_train, brand_col, model_col, pairs_list, splits
).rename(columns={
    "MAE_mean": "MAE_mean_global", "MAE_std": "MAE_std_global",
    "RMSE_mean": "RMSE_mean_global", "RMSE_std": "RMSE_std_global",
    "n": "n_global"
})

df_pair_spec = eval_pair_specific(
    pipe_template, X_train, y_train, brand_col, model_col, pairs_list, splits, min_train_per_fold=40
).rename(columns={
    "MAE_mean": "MAE_mean_pair", "MAE_std": "MAE_std_pair",
    "RMSE_mean": "RMSE_mean_pair", "RMSE_std": "RMSE_std_pair",
    "n": "n_pair"
})

df_compare_pair = df_global_pair.merge(df_pair_spec, on=["brand", "model"], how="inner")
df_compare_pair["delta_MAE"] = df_compare_pair["MAE_mean_pair"] - df_compare_pair["MAE_mean_global"]
df_compare_pair["delta_RMSE"] = df_compare_pair["RMSE_mean_pair"] - df_compare_pair["RMSE_mean_global"]

df_compare_pair = df_compare_pair.sort_values("delta_MAE")

# Full results (all frequent pairs) are here:
display(df_compare_pair)

# For the report: show the most improved + most harmed (readable subset)
display_cols = ["brand", "model", "n_global", "MAE_mean_global", "MAE_mean_pair", "delta_MAE",
                "RMSE_mean_global", "RMSE_mean_pair", "delta_RMSE"]

print("Top 15 improvements (most negative ΔMAE):")
display(df_compare_pair[display_cols].head(15).round(1))

print("Top 15 degradations (most positive ΔMAE):")
display(df_compare_pair[display_cols].tail(15).round(1))

# Plots (same as before): ΔMAE bar plot for stable segments + scatter vs size
MIN_PLOT_SAMPLES = 100
df_plot = df_compare_pair[df_compare_pair["n_global"] >= MIN_PLOT_SAMPLES].copy()

plt.figure(figsize=(10, 4))
x = np.arange(len(df_plot))
plt.bar(x, df_plot["delta_MAE"])
plt.axhline(0, linestyle="--")
plt.xticks(
    x,
    [f"{b} {m}" for b, m in zip(df_plot["brand"], df_plot["model"])],
    rotation=90, ha="right"
)
plt.ylabel("Δ MAE (pair - global)")
plt.title("Effect of (brand, model) specialization (negative = improvement)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
plt.scatter(df_compare_pair["n_global"], df_compare_pair["delta_MAE"])
plt.axhline(0, linestyle="--")
plt.xlabel("Number of samples per (brand, model) (out-of-fold counted)")
plt.ylabel("Δ MAE (pair - global)")
plt.title("ΔMAE vs segment size")
plt.tight_layout()
plt.show()

# Long Duration (~5min)

#### 4.3 Price Predictor Web Application

The `ML Cars 4 You` Web Application is implemented in `app.py` and hosted live: 

👉 **[Click here to open the ML Cars 4 You App](https://ml-cars-4-you.streamlit.app)**



#### a.) Objective and Motivation

**Project Goal**  

The objective of this web application is to provide an accessible, user-friendly interface for predicting used car prices based on our best machine learning model. This deployment serves as the practical implementation component of the Open End Section, transforming our trained model into a production-ready tool.

**Motivation**
- **Accessibility**: Make the ML model accessible to non-technical users through an intuitive web interface
- **User Experience**: Provide instant price predictions with intervals to help users make informed decisions

**Target Users**
- Used car buyers seeking fair price estimates
- Car dealers looking for market-competitive pricing

---

#### b.) Difficulty of Tasks

1. Model Size Management (High Difficulty)
- **Challenge**: The trained model file (`rf_tuned_pipe.pkl`) was 900 mb, far exceeding GitHub's 100 MB limit
- **Solution**: Implemented Git Large File Storage (LFS) to handle the massive model file
- **Complexity**: Required understanding of Git LFS, bandwidth limitations, and deployment constraints

2. Dynamic Feature Engineering (Medium Difficulty)
- **Challenge**: The model uses custom preprocessing functions from `pipeline_functions.py`
- **Solution**: Ensured all custom transformers and functions are properly included in the repository
- **Complexity**: Maintaining consistency between training pipeline and deployment environment

3. Brand-Model Dependency Mapping (Medium-Low Difficulty)
- **Challenge**: Creating a user-friendly interface where model selection dynamically updates based on brand choice
- **Solution**: Implemented a nested dictionary structure with 9 brands and 114 models
- **Complexity**: Balancing comprehensive coverage with UI simplicity
---


#### c.) Correctness and Efficiency of Implementation

**Accurate Prediction Pipeline**
- The `load_model()` function correctly loads the sklearn pipeline
- Input data is formatted as a pandas DataFrame with exact feature names and types expected by the model

**Input Validation**
- Realistic min/max values based on dataset ranges (e.g., year: 1970-2020)
- Step increments appropriate for each input type
- Default values represent typical vehicles

**Performance Optimizations**
- `@st.cache_resource` decorator caches the model in memory (loads only once)
- Single prediction call per button press (no redundant computations)

---
#### d.) Discussion of Results

This web application successfully bridges the gap between ML model development and real-world deployment. Despite technical challenges, the implementation achieves its core objectives:

- **Accessible**: Anyone can use the tool without ML knowledge  
- **Accurate**: MAE of £1,200 provides actionable price estimates  
- **Efficient**: Cached model and optimized code ensure fast predictions  

The deployment demonstrates practical ML engineering skills including version control (Git LFS), dependency management, cloud deployment (Streamlit Cloud), and user-centric design.  


However, this tool has important limitations:

- **Brand Bias**: Performance is strongest on well-represented brands while predictions for underrepresented manufacturers may be less reliable
- **Temporal Relevance**: The model was trained on data up to 2020 and may not fully capture recent market dynamics such as semiconductor shortages or accelerated EV adoption
- **Edge Cases**: Predictions for rare models, unusual configurations, or vehicles at price extremes should be treated with additional caution

### V. Deployment and Prediction on Test Set

This section of the notebook includes the deployment to generate reliable predictions for new data. The pipeline is stored in the `et_tuned_pipe.pkl` file and the final output of the predicted test data is stored in `Group05_Version20.csv` (selected as best on Kaggle).

In [ ]:
joblib.dump(et_tuned_pipe, "et_tuned_pipe.pkl")

In [ ]:
def predict_on_test(model_pipeline, model_name):
    # Load best model from Joblib and predict on validation set to verify
    pipe_best = joblib.load(model_pipeline)
    
    # Predict on test set
    df_cars_test['price'] = pipe_best.predict(df_cars_test)
    df_cars_test[['carID', 'price']].to_csv(f'Group05_Version20.csv', index=False)

In [ ]:
predict_on_test("et_tuned_pipe.pkl", "ET")

In [ ]:
# !kaggle competitions submit -c cars4you -f Group05_Version05.csv -m "Message" # Uncomment to submit to Kaggle

In [ ]:
# !kaggle competitions submissions -c cars4you

**Findings:**     
The test score is close to validation score so our model should generalize well, assuming that the data in validation and test is similar.

### VI. Discussion and Outlook

**Tree-based models performed best:**     
Regarding the baseline models we used in Section 5, it becomes clear that the tree-based models outperform the other models. This is probably due to ...

**Constraints:**     
However, the nature of tree-based models constraints the predictions to never be lower than the lowest or higher than the highest price in the train set. This is because tree-based models return the average price of the cars in the leaf node.

**Outlook:**     
A potential solution of this could be a **Stacking Regressor** that combines a tree-based model with another model that is better in extrapolation (e.g. RF + Linear Regression). A final Meta-Learner (also Linear) can combine their predictions. If the RF predicts "Max Value" but the Linear Model predicts "Higher Value," the Meta-Learner could follow the Linear trend upward.    